In [2]:
#!/usr/bin/env python

import os
import heapq
import numpy as np
import hnswlib
import time
import matplotlib.pyplot as plt
from connected_components import group_indices
from collections import Counter
from multiprocessing import Pool

KUPPA = 0.4 # rough idea of how many density peaks given C total contigs
TOTAL_CONTIGS = 0
LATENT_DIMENSION = 0
ETA = 0 # 1/latent_dim weight of contigs length
DISTANCE_CUTOFF = 1
NN_SIZE_CUTOFF = 200000
NCPUS = (os.cpu_count()-2 if os.cpu_count() is not None else 8)

def add_selfindex(data_indices, labels, distances):
    """ add self index where missed by hnswlib """
    s2 = time.time()
    # in case query index does not found as first nearest neighbor,
    # insert manually and remove last nearest neighbor
    add_selfinds = np.nonzero(labels.astype(int)[:,0] != data_indices)[0]
    labels[add_selfinds] = np.insert(labels[add_selfinds], 0, add_selfinds, axis=1)[:,:-1]
    distances[add_selfinds] = np.insert(distances[add_selfinds], 0, 0.0, axis=1)[:,:-1]
    print(time.time()-s2, 'self index adding')
    return labels, distances

def get_neighbors(latent, length):
    """ Get neighbors and distance cutoff"""
    s= time.time()
    data_indices = np.arange(TOTAL_CONTIGS)
    k_nn = int(TOTAL_CONTIGS ** KUPPA)

    p = hnswlib.Index(space = 'l2', dim = LATENT_DIMENSION)
    # M - max. number of out-edges ef_construction
    # ef_construction - the size of the dynamic list for the nearest neighbors
    # and controls index_time/accuracy. Important during indexing time
    p.init_index(max_elements = TOTAL_CONTIGS, ef_construction = 200, M = 16)
    p.set_ef(k_nn) # number should be greater than k_nn
    p.set_num_threads(NCPUS)
    p.add_items(latent)

    retry = True
    while retry:
        labels, distances = p.knn_query(latent, k=k_nn)
        cumsum_length = np.cumsum(length[labels], axis=1)
        contigs_with_enough_neighbors = np.nonzero(cumsum_length[:,-1] > NN_SIZE_CUTOFF)[0]
        retry = len(contigs_with_enough_neighbors) / TOTAL_CONTIGS < 0.5
        if retry:
            k_nn *= 2 # adjustable
            p.set_ef(k_nn) # adjustable

    labels, distances = add_selfindex(data_indices, labels, distances)
    dist_cutoff_indices = np.argmax(
            np.cumsum(length[labels], axis=1) > NN_SIZE_CUTOFF, axis=1)
    indices_tofind_dist = np.nonzero(dist_cutoff_indices)[0]
    # for long contigs dist_cutoff_indices would be zero. Hence, add manually
    ind_check_longcontig = np.nonzero(dist_cutoff_indices==0)[0]
    indices_to_add = np.nonzero(length[ind_check_longcontig]> NN_SIZE_CUTOFF)[0]
    indices_tofind_dist = np.concatenate((indices_tofind_dist, \
                                        ind_check_longcontig[indices_to_add]))
    
    # Find distance cutoff
    global DISTANCE_CUTOFF
    DISTANCE_CUTOFF = np.median(
        distances[indices_tofind_dist,dist_cutoff_indices[indices_tofind_dist]])
    print(DISTANCE_CUTOFF, np.sqrt(DISTANCE_CUTOFF), 'distance cutoff')

    valid_indices = [[] for _ in range(TOTAL_CONTIGS)]
    
    # flags to indicate contigs found all nearest neighbors within DISTANCE_CUTOFF
    flags = distances[:,-1] > DISTANCE_CUTOFF
    c_indices = np.where(flags==1)[0]

    def update_nnlists(c_indices, nn_lists, nn_dists):
        # Create a mask for distances less than DISTANCE_CUTOFF
        mask = nn_dists <= DISTANCE_CUTOFF
        # Apply the mask to nn_inds_k
        filtered_inds = np.where(mask, nn_lists, -1).astype(int)  # Using -1 as a placeholder for invalid indices
        for num, i in enumerate(c_indices):
            valid_indices[i] = filtered_inds[num, filtered_inds[num] >= 0]

    update_nnlists(c_indices, labels[flags], distances[flags])

    k_nn_more = k_nn
    remaining_indices = np.where(flags==0)[0]
    
    while len(remaining_indices) > 0:
        k_nn_more *= 2
        p.set_ef(k_nn_more)
        nn_lists, nn_dists = p.knn_query(latent[remaining_indices], k = k_nn_more)
        flags_check = nn_dists[:,-1] > DISTANCE_CUTOFF
        indices_toupdate = np.where(flags_check==1)[0]
        # if indices_toupdate.size > 0:
        if k_nn_more < 10000 and indices_toupdate.size > 0:
            c_indices = remaining_indices[indices_toupdate]
            remaining_indices = remaining_indices[~flags_check]
            update_nnlists(c_indices, nn_lists[indices_toupdate], nn_dists[indices_toupdate])
        else:
            # stop if k_nn_more increases to impede knn_query and \
            # compute distance with all points for remaining contigs to \
            # valid indices
            distances_remaining = np.sum((latent - latent[remaining_indices][:,np.newaxis])**2,axis=2)
            for row_distance, inds in zip(distances_remaining, remaining_indices):
                nn_inds = np.argsort(row_distance)
                nn_inds = nn_inds[np.where(row_distance[nn_inds] <= DISTANCE_CUTOFF)]
                valid_indices[inds] = nn_inds
            break
    
    # for query_point in indices_getmorenn:
    #     k_nn_more = k_nn
    #     while True:
    #         indices, dist = p.knn_query(latent[query_point], k = k_nn_more)
    #         if dist[0][-1] > DISTANCE_CUTOFF:
    #             labels_dict[query_point] = indices[0][np.where(dist[0] <= DISTANCE_CUTOFF)[0]]
    #             break
    #         else:
    #             k_nn_more *= 2

    # print(labels_dict)

    # print(len(flags), TOTAL_CONTIGS - len(flags))
    # print(len(distances[~flags][:,-1]), len(distances[flags][:,-1]), 'distance in the flag')
    # print(sum(flags))
    # get all neighbors within distance cutoff
    # add_moreknn = np.min(distances[:,-1]) < DISTANCE_CUTOFF
    # while add_moreknn:
    #     k_nn *= 3 # adjustable
    #     p.set_ef(k_nn)
    #     labels, distances = p.knn_query(latent, k=k_nn+1)
    #     add_moreknn = np.min(distances[:,-1]) < DISTANCE_CUTOFF
    # labels, distances = add_selfindex(data_indices, labels, distances)
    # print(time.time()-s ,'seconds for knn')
    # del p

    return valid_indices, labels, distances

def get_density_peaks(valid_indices, labels, distances, length):
    """ compute density using k-nearest neighbors """
    s = time.time()

    # # Create a mask for distances less than DISTANCE_CUTOFF
    # mask = distances <= DISTANCE_CUTOFF

    # # Apply the mask to nn_inds_k
    # filtered_inds = np.where(mask, labels, -1).astype(int)  # Using -1 as a placeholder for invalid indices
    # del mask
    # valid_indices = [filtered_inds[c, filtered_inds[c] >= 0] for c in range(TOTAL_CONTIGS)]
    # del filtered_inds

    densities = np.array([np.sum(length[valid_indices[c]] ** ETA) / DISTANCE_CUTOFF for c in range(TOTAL_CONTIGS)])
    
    graph = {}
    nearest = np.arange(TOTAL_CONTIGS)
    density_peaks_flag = np.full(TOTAL_CONTIGS,-1)
    
    for c, valid_inds in enumerate(valid_indices):
        # single member clusters
        if valid_inds.size == 1:
            # print(densities[valid_inds], 'densities of single member', c)
            if densities[valid_inds] >= NN_SIZE_CUTOFF: # long genomic contig as a single cluster
                density_peaks_flag[c] = c
                graph[c] = []
                nearest[c] = c
            else: # short isolated contig
                # maxdens_neighbor = np.argmax(densities[labels[c]])
                # if distances[c][maxdens_neighbor] <= DISTANCE_CUTOFF * 1.5 :
                #     graph[c] = labels[c][maxdens_neighbor]
                #     nearest[c] = labels[c][maxdens_neighbor]
                # else:
                #     graph[c] = []
                #     nearest[c] = labels[c][1]

                # HyperParam 1.8
                nearest_points = np.nonzero(distances[c] <= DISTANCE_CUTOFF * 1.8)[0]
                if nearest_points.size > 1:
                    near_pointdensities = densities[labels[c][nearest_points]]
                    # Maybe applicable only for Toy datasets with counts as densities
                    # if np.all(near_pointdensities == near_pointdensities[0]):
                    #     graph[c] = labels[c][1]
                    #     nearest[c] = labels[c][1]
                    # else:
                    maxdens_neighbor = nearest_points[np.argmax(near_pointdensities)]
                    graph[c] = labels[c][maxdens_neighbor]
                    nearest[c] = labels[c][maxdens_neighbor]
                else:
                    graph[c] = []
                    nearest[c] = labels[c][1]

            continue
        
        max_densityindex = valid_inds[np.argmax(densities[valid_inds])]
        # valid_inds all can have the same density but small. They become separate low-density peaks
        # TODO: have to solve to merge such low-density peaks with higher density peaks
        if c == max_densityindex:
            density_peaks_flag[c] = c
            nearest[c] = c
        else:
            nearest[c] = max_densityindex
            
        graph[c] = list(zip(valid_inds[1:], densities[valid_inds[1:]]))

    print(time.time() - s, 'seconds for graph and densities calculation')
    density_peaks_inds = np.nonzero(density_peaks_flag>=0)[0]
    
    peak_count = int(TOTAL_CONTIGS ** (1-KUPPA))

    if peak_count > len(density_peaks_inds): 
        peak_count = len(density_peaks_inds)
    
    get_inds = np.argsort(densities[density_peaks_inds])[::-1][:peak_count]
    density_peaks = density_peaks_inds[get_inds]

    # plt.figure(figsize=(20,12))
    # plt.plot(densities)

    return densities, density_peaks, graph, nearest

def dijkstra_max_min_density(graph, start):
    """ return max density path between start and nearest neighbors """    
    
    # Initialize maximum minimum densities and priority queue
    max_min_densities = {node: float('-inf') for node in graph}
    max_min_densities[start] = float('inf')
    priority_queue = [(-max_min_densities[start], start)]  # Priority queue of (negative density, node)

    while priority_queue:
        # Extract node with highest maximum minimum density
        current_density, current_node = heapq.heappop(priority_queue)
        current_density = -current_density

        for neighbor, density in graph[current_node]:
            # Calculate new maximum minimum density for neighbor
            new_density = min(current_density, density)
            if new_density > max_min_densities[neighbor]:
                max_min_densities[neighbor] = new_density

                heapq.heappush(priority_queue, (-new_density, neighbor))

    return max_min_densities

def find_connected_components(merge_links):

    K = len(merge_links)
    merge_sets = np.zeros(K).astype(int) - 1
    merge_curr = 0
    for k in np.arange(K):

        if merge_sets[k] < 0:

            candidates = merge_links[k]

            merge_sets[k] = merge_curr

            while len(candidates) != 0:
                l = candidates.pop(-1)

                if merge_sets[l] < 0:
                    merge_sets[l] = merge_curr
                    candidates.extend(merge_links[l])

            merge_curr += 1

    return merge_sets

def remove_outliers_peaks(data, m=2.):
    d = np.abs(data - np.median(data))
    mdev = np.median(d)
    s = d / (mdev if mdev else 1.)
    return np.where(s < m)[0] # indices of outlier points

def density_links(args):
    i, peak, graph, density_peaks, densities = args
    print(peak, i, 'peak and i')
    max_min_densities = dijkstra_max_min_density(graph, peak)
    higherdensity_links = {k:v for k, v in max_min_densities.items() if (k in density_peaks) and (k != peak) and (v != float('-inf'))}
    if higherdensity_links:
        max_key = max(higherdensity_links, key=higherdensity_links.get)
        separability_index = 1 - (higherdensity_links[max_key] / densities[peak])
        merge_peakinds = np.where(np.isin(density_peaks, list(higherdensity_links.keys())))[0]
        merge_link = [i] + list(merge_peakinds)
    else:
        separability_index = 1
        merge_link = []
    return i, separability_index, merge_link


def cluster(latent, contig_length):
    """ cluster contigs """

    valid_indices, labels, distances = get_neighbors(latent, contig_length)
    print(valid_indices, 'labels and distances')
    densities, density_peaks, graph, nearest = get_density_peaks(valid_indices, labels, distances, contig_length)
    print(len(density_peaks), 'total density peaks')
    separability_indices = np.full(len(density_peaks), 1.0, dtype=np.float32)

    # peak_neighbors = [valid_indices[i].tolist() for i in density_peaks]
    # print(graph, 'graph')
    
    # connected_peakcomponents = group_indices(peak_neighbors)
    
    # # from connected_components import select_subgraph
    # # subgraph = select_subgraph(graph, connected_peakcomponents[0][1])
    # # print(subgraph, 'subgraph')
    # subgraph1 = {node: [adj_node for adj_node in graph[node] if adj_node in connected_peakcomponents[0][1]] for node in connected_peakcomponents[0][1]}
    # print(subgraph1, 'subgraph1')
    # merge_links = []
    # for i, peak in enumerate(density_peaks):
    #     max_min_densities = dijkstra_max_min_density(graph, peak)

    #     higherdensity_links = {k:v for k, v in max_min_densities.items() if (k in density_peaks) and (k != peak) and (v != float('-inf'))}
    #     # higherdensity_links = {k:v for k, v in higherdensity_links.items() if v > densities[peak]} # this doesn't work when v and densities[peak] is same value
    #     if higherdensity_links:
    #         # print(peak, higherdensity_links)
    #         max_key = max(higherdensity_links, key=higherdensity_links.get)
    #         separability_indices[i] = 1 - (higherdensity_links[max_key] / densities[peak])
    #         merge_peakinds = np.where(np.isin(density_peaks, list(higherdensity_links.keys())))[0]
    #         merge_links.append([i] + list(merge_peakinds))
    #     else:
    #         merge_links.append([])

    # # args_list = [(i, peak, graph, density_peaks, densities) for i, peak in enumerate(density_peaks)]
    # # merge_links = []

    # # with Pool(processes=NCPUS) as pool:
    # #     results = pool.map(density_links, args_list)

    # # for i, separability_index, merge_link in results:
    # #     separability_indices[i] = separability_index
    # #     merge_links.append(merge_link)
    
    # # # not useful for deciding number of clusters and assignment as we have merging of density peaks
    # plt.figure(figsize=(16,12))
    # plt.scatter(densities[density_peaks], separability_indices)
    # plt.xlabel('density')
    # plt.ylabel('separability index')

    # separability_indices = np.sort(separability_indices)[::-1]

    # cluster_centercounts = np.argmax(np.abs(np.diff(separability_indices))) + 1
    # if cluster_centercounts == 1:
    #     cluster_centercounts = len(density_peaks)
   
    # # # cluster_centers = density_peaks[densities[density_peaks] > int(TOTAL_CONTIGS ** KUPPA)] # density_peaks[:cluster_centercounts]
    # # ##################################
    
    # merge_sets = find_connected_components(merge_links)

    # return densities, density_peaks, merge_sets, nearest, cluster_centercounts


def assign_points(density_peaks, merge_sets, nearest, densities):
    """ assign points to density peaks and merge closer density peaks """
    nearest[density_peaks] = density_peaks
    
    nearest_prev = np.zeros(nearest.size, dtype=int) - 1

    while (nearest != nearest_prev).any():
        nearest_prev = nearest
        nearest = nearest[nearest[nearest[nearest[:]]]]

    components = []
 
    for k in range(len(density_peaks)):
        
        if len((np.argwhere(nearest == density_peaks[k])[0])) > 0:
            components.append(np.nonzero(nearest == density_peaks[k])[0])

        else:
            raise RuntimeWarning("no clusters in cluster_centers[k] is assigned to nearest[k]")

    clusters = []

    unassigned_points = np.setdiff1d(nearest, density_peaks)
    # print(unassigned_points, labels[unassigned_points], 'assignment step unassigned points')

    for i in set(merge_sets):
        merge_indices = np.nonzero(merge_sets == i)[0]
        merge_temp = []
        for j in merge_indices:
            merge_temp.extend(components[j])
        clusters.append(merge_temp)
    
    # for f in clusters:
    #     print(len(f), sum(densities[f]))
        
    return clusters, components

if __name__ == "__main__":
  
    variable = '/Users/yazhini/Documents/work/'
    latent = np.load(variable + 'latent_mu.npy')
    contig_length = np.load(variable + 'contigs_2klength.npz',allow_pickle=True)['arr_0']
    # contig_names = np.load(variable + 'contigs_2knames.npz',allow_pickle=True)['arr_0']
    otuids = np.loadtxt(variable + 'otuids', dtype='object')
    
    TOTAL_CONTIGS, LATENT_DIMENSION = latent.shape
    ETA = 0.4 # 1 / LATENT_DIMENSION

    print(latent.shape)
    cluster(latent, contig_length)
    # densities, density_peaks, merge_sets, nearest, cluster_centercounts = cluster(latent, contig_length)
    # clusters, components = assign_points(density_peaks, merge_sets, nearest, densities)
    # labels = np.zeros(TOTAL_CONTIGS,dtype=int)

    # counter = 1
    # for i in range(len(clusters)):
    #     labels[clusters[i]] = counter
    #     counter += 1

    # for j in range(len(labels)):
    #     print(labels[j], otuids[j])

    # import seaborn as sns
    # import plotly.express as px
    # fig = px.scatter(x=latent[:,8], y=latent[:,4])
    # fig.update_traces(marker=dict(size=1))
    # fig.show()



(159956, 32)
0.012894392013549805 self index adding
19052.312 138.03012 distance cutoff
(159956, 120) (159956, 120) labels and distances
4.598473072052002 seconds for graph and densities calculation
1325 total density peaks


In [33]:
([0], [38971, 87811, 81521, 97027, 74112, 131079, 62297, 93367, 1013, 65612, 112246, 121827, 149344, 95280, 150900, 380, 64272, 643, 91400, 35990, 145304, 43235, 25309, 64044, 61203, 43819, 139042, 122989, 93621, 103778, 80499, 153003, 146715, 72916, 159930, 64572, 113774, 127623, 99045, 145104, 11849, 7803, 8163, 46640, 116874, 29446, 123727, 83132, 12455, 79540, 40610, 114171, 39328, 103315, 112849, 130591, 65089, 41798, 62131, 107306, 91572, 25419, 39319, 158318, 6414, 149467, 128474, 73734, 64708, 44097, 126162, 108953, 129816, 75994, 24728, 33343, 50696, 26320, 90777, 151794, 82508, 149984, 26664, 75026, 52463, 110336, 12476, 57923, 127122, 108628, 106114, 81570, 38123, 99056, 116509, 35934, 80356, 144382, 153772, 43401, 158472, 114297, 83250, 117777, 157426, 29562, 66012, 106541, 105762, 53400, 1342, 76942, 1708, 48391, 58797, 17796, 135181, 149503, 148859, 47945, 108363, 95099, 65927, 94086, 126906, 96248, 52400, 30468, 25291, 13358, 14245, 155209, 91448, 45554, 62602, 87671, 133282, 120531, 18727, 30260, 25945, 53759, 113903, 130119, 84239, 65820, 41719, 21031, 28707, 112284, 93876, 9814, 121741, 83793, 17754, 156034, 71437, 121720, 140785, 100952, 152834, 152774, 117412, 128025, 124254, 27534, 27689, 42295, 149938, 158306, 48411, 29135, 146176, 29715, 58258, 54871, 55025, 62885, 61519, 140385, 119458, 7177, 47965, 90523, 77021, 67049, 39803, 140955, 31039, 127391, 19776, 55235, 150507, 72590, 37755, 42558, 100636, 24311, 147185, 64690, 105299, 29070, 69229, 20684, 30369, 39519, 131943, 19034, 110807, 25639, 35176, 60133, 72477, 33876, 147583, 142616, 52519, 19484, 134390, 130598, 87082, 149372, 20488, 10109, 87447, 50091, 145954, 38272, 11499, 133043, 118912, 8476, 7651, 29894, 47573, 14687, 128819, 73557, 56250, 141916, 29795, 28391, 103944, 28277, 22590, 50628, 69646, 25010, 16160, 31945, 28209, 105799, 149667, 51293, 69980, 102931, 51666, 48408, 128005, 134719, 55115, 71388, 11120, 2048, 89893, 20418, 125227, 18257, 123693, 124156, 16390, 55342, 13822, 10162, 143594, 48798, 131978, 74038, 48781, 13840, 105245, 134309, 123058, 47485, 56317, 125048, 157868, 47149, 105867, 60625, 33671, 94841, 128205, 54675, 151855, 147578, 126090, 25699, 136091, 82357, 83951, 58561, 70578, 107739, 20241, 153533, 15044, 24139, 58278, 18999, 153863, 12253, 14448, 91288, 88449, 90759, 28506, 108932, 14808, 150115, 125619, 40106, 30015, 92844, 108950, 44252, 137529, 142755, 60811, 43901, 143345, 159215, 145907, 2602, 23708, 149922, 7961, 58485, 31638, 73726, 133084, 129725, 129856, 121130, 141854, 23016, 17520, 21507, 83281, 140807, 104179, 104978, 17256, 139548, 126866, 90904, 146400, 63933, 37730, 14510, 10705, 147287, 38506, 33410, 61001, 18179, 88236, 49418, 92289, 95865, 142186, 29890, 88831, 130886, 116526, 3970, 9386, 47584, 27280, 121039, 89384, 121313, 137773, 83738, 73130, 152670, 124402, 3024, 151938, 96405, 16792, 67528, 24989, 89662, 146177, 13555, 84356, 142908, 114654, 7632, 66063, 68268, 146598, 158059, 139653, 84580, 73580, 41491, 87446, 122678, 117590, 84959, 33073, 114368, 20222, 122396, 20081, 39873, 145789, 76740, 87068, 41342, 66690, 58369, 78468, 86190, 33311, 106651, 63677, 48992, 124806, 60954, 75759, 52852, 94790, 123355, 146856, 48063, 121816, 100368, 86670, 121169, 1926, 23454, 135752, 15592, 151258, 41377, 125523, 90201, 81232, 74278, 44821, 137601, 34556, 113639, 107760, 142695, 5827, 96227, 125683, 75295, 52896, 148515, 77684, 159605, 96121, 8448, 126, 129406, 64700, 66978, 157907, 43625, 2633, 23172, 78948, 156193, 88005, 159021, 20090, 150098, 159942, 59326, 96788, 95534, 70948, 151830, 9630, 95234, 113538, 157053, 69035, 91621, 59647, 1918, 34262, 91076, 159874, 118354, 137915, 74783, 112092, 61171, 157733, 112042, 54960, 121585, 75192, 22331, 128792, 98013, 150883, 115038, 65121, 111167, 59010, 24974, 33605, 118036, 101282, 1299, 45962, 26309, 59463, 73035, 25231, 53528, 68644, 83778, 53718, 78642, 50007, 83949, 82067, 137599, 2014, 30816, 149393, 82158, 64881, 21222, 59986, 44178, 65544, 81926, 65784, 28493, 103799, 130199, 110709, 26125, 106443, 46613, 115148, 30211, 59316, 134107, 3417, 4118, 106587, 116997, 74721, 107322, 148188, 117138, 117083, 64628, 103753, 144549, 80267, 30796, 38111, 120402, 74446, 98003, 54698, 36476, 159203, 57479, 58035, 72472, 111427, 52768, 74854, 41315, 49319, 90982, 76628, 17583, 503, 29343, 119942, 102581, 41068, 24480, 51618, 58738, 55590, 137010, 158409, 1692, 141758, 153009, 152685, 24710, 110330, 91848, 7146, 109543, 133590, 24407, 79393, 34484, 157045, 96981, 84813, 53396, 148553, 135894, 100261, 91918, 73827, 96309, 41887, 117442, 73901, 61415, 138833, 18684, 804, 95557, 156325, 60600, 65857, 10338, 57866, 13280, 10057, 100219, 49698, 56030, 115738, 14746, 9773, 151511, 70270, 38263, 60170, 45439, 138665, 2307, 125657, 39498, 111345, 139295, 22411, 20745, 29912, 17022, 139911, 68014, 147107, 84812, 73894, 25242, 68327, 104083, 122870, 61076, 45806, 122168, 77417, 37055, 85321, 88817, 8985, 63679, 107396, 141990, 69764, 91508, 35204, 83303, 92732, 36737, 8870, 6438, 129386, 35224, 99308, 12411, 156129, 86223, 39908, 120148, 123545, 73778, 34510, 98079, 23431, 49749, 28330, 135421, 35511, 154024, 53148, 19751, 2395, 87509, 84726, 29079, 32030, 95266, 146137, 138323, 113218, 15546, 2923, 127064, 92125, 56172, 7927, 93638, 34672, 145417, 93850, 120187, 95404, 87588, 66462, 118879, 4779, 104428, 62852, 151633, 72710, 31364, 2248, 17063, 41528, 58349, 25681, 72085, 134233, 8984, 76084, 21531, 132959, 39681, 45045, 143790, 81631, 16698, 12796, 60877, 12012, 108247, 87499, 107948, 128174, 135641, 91474, 16839, 145469, 129520, 66086, 7298, 155916, 23635, 7586, 138630, 123706, 68467, 100548, 23157, 85431, 48035, 81217, 5956, 149837, 65208, 20749, 118353, 150325, 134397, 68213, 94154, 18476, 100341, 37349, 79819, 98095, 120706, 155663, 75019, 50503, 79062, 19231, 13450, 54094, 154720, 66052, 86638, 46967, 37481, 153888, 149594, 10972, 126618, 87287, 64442, 95446, 21887, 134710, 96740, 119818, 98508, 36149, 30830, 17423, 51906, 97875, 73600, 95916, 130548, 19448, 5233, 155063, 75877, 131025, 118756, 150714, 103169, 115164, 156976, 80469, 75038, 156212, 91870, 127522, 129421, 44143, 42213, 35250, 115957, 16940, 18540, 151530, 63467, 61621, 131193, 46485, 101262, 58765, 149301, 124399, 25384, 45536, 95830, 64490, 136792, 17021, 313, 132138, 97023, 2474, 156934, 35375, 55257, 94461, 13918, 155282, 47183, 49557, 70263, 36185, 89088, 89345, 89575, 4434, 76888, 106313, 129146, 40198, 764, 52193, 87310, 58890, 75657, 94554, 145468, 155368, 52105, 64962, 78814, 61771, 139330, 87591, 116454, 54996, 70278, 87798, 145611, 125904, 127596, 56853, 44764, 138464, 428, 94269, 90090, 81060, 53332, 76464, 83332, 129916, 134658, 298, 122260, 61258, 61179, 43735, 412, 60706, 45905, 127040, 65163, 106519, 5677, 116667, 68680, 23686, 51197, 100088, 143705, 31825, 59426, 91342, 83001, 50831, 155425, 120126, 145855, 45654, 120536, 20050, 99051, 149369, 92354, 88166, 32939, 114201, 127285, 66785, 50384, 61744, 21122, 29027, 37401, 27574, 125375, 11331, 116029, 36351, 67042, 9861, 57771, 155698, 39412, 34558, 139307, 117537, 104834, 31887, 26241, 138114, 39173, 51534, 15089, 1086, 114748, 34245, 43028, 43510, 31332, 47039, 69101, 101820, 60947, 156511, 127930, 119616, 128892, 40220, 47643, 136116, 16438, 18605, 28801, 18053, 147742, 82670, 22959, 113091, 86795, 130579, 137275, 117163, 58358, 39676, 56116, 112679, 153520, 66712, 152069, 156340, 9270, 114267, 60658, 59659, 4161, 130117, 95015, 63974, 118243, 101260, 91524, 81517, 25210, 154777, 58463, 129459, 76906, 60286, 116096, 72389, 75723, 13745, 69369, 109246, 94889, 71763, 67680, 49911, 142095, 126374, 32508, 145091, 109307, 44327, 78559, 141445, 65161, 43433, 55346, 8157, 140055, 11079, 113233, 50000, 136753, 19882, 145234, 11949, 138555, 55403, 151163, 57247, 125595, 58614, 103627, 116646, 44016, 158737, 71355, 108472, 92805, 103529, 83633, 13808, 116558, 7634, 100285, 129572, 9658, 118770, 52616, 135286, 129926, 78347, 144444, 84738, 5456, 103621, 150338, 94098, 37262, 92882, 12309, 22686, 75314, 113821, 83419, 26358, 31338, 120090, 146871, 86541, 60867, 44613, 40430, 88812, 134521, 423, 124710, 135346, 102680, 57516, 99460, 131011, 14257, 78144, 74205, 29754, 122412, 118477, 148871, 105657, 128662, 124227, 66575, 90598, 48458, 140060, 146611, 6298, 19840, 140109, 146275, 117487, 68224, 120711, 46216, 159344, 71055, 812, 20104, 75557, 106543, 4999, 107938, 24019, 153390, 25964, 88002, 10269, 2335, 84889, 61117, 48465, 49209, 35030, 98083, 33856, 54216, 134906, 46204, 37359, 32238, 32001, 77443, 32994, 149934, 10472, 38794, 125396, 73793, 42592, 19950, 121683, 142385, 87230, 28164, 89785, 103431, 94115, 57427, 73069, 26367, 37955, 133823, 108749, 40107, 157591, 150992, 10288, 85279, 65986, 21682, 124676, 38301, 89557, 118656, 30178, 129223, 76427, 114348, 103256, 97794, 99648, 131988, 65778, 42997, 159578, 138246, 82295, 68893, 27538, 8502, 136674, 129748, 77084, 123481, 48887, 57819, 41101, 13414, 143634, 63835, 30126, 86570, 102809, 55284, 119678, 129697, 54868, 158084, 56099, 89542, 15452, 131349, 129940, 76309, 72413, 116322, 103197, 59320, 113269, 23885, 25446, 64038, 64786, 153019, 129463, 53016, 98267, 45907, 141592, 101604, 69314, 132144, 109774, 87857, 105010, 150742, 71575, 155670, 35470, 2519, 104832, 56741, 2856, 115215, 4262, 60251, 26539, 79787, 134543, 44873, 49582, 117486, 149668, 34610, 14498, 20092, 91453, 33110, 159260, 126065, 133733, 85485, 125935, 129526, 52618, 114633, 83209, 12135, 151253, 40868, 59634, 41034, 38861, 28767, 60747, 56711, 154298, 140801, 53567, 92744, 112296, 53971, 75431, 159222, 157013, 14101, 136355, 65432, 22521, 52432, 115937, 148884, 71788, 83861, 119664, 118917, 77153, 2695, 147951, 73266, 75074, 150574, 135986, 73473, 49327, 101165, 25338, 7789, 14033, 136299, 56247, 116183, 23090, 42335, 61574, 49326, 7776, 107868, 56380, 1477, 127738, 51871, 61814, 46684, 38080, 154876, 48629, 3447, 8922, 65191, 112746, 46492, 22772, 7090, 17072, 3064, 115152, 80726, 84300, 35594, 99538, 114447, 31606, 46074, 26581, 159708, 22886, 102189, 62702, 133296, 30240, 65950, 106447, 11482, 66092, 138917, 33775, 151341, 136329, 46953, 18024, 45647, 129429, 103055, 17672, 97740, 35927, 53645, 22525, 40813, 106906, 119005, 152237, 72285, 134473, 44963, 130650, 18706, 115259, 62222, 60985, 96311, 39924, 105240, 151212, 81091, 146829, 131044, 70140, 25606, 65615, 115123, 14889, 80760, 95096, 44313, 70775, 120419, 98982, 69497, 138731, 84855, 53478, 155000, 121506, 145177, 101894, 1551, 73322, 87448, 152235, 100451, 158867, 70478, 43250, 94168, 127098, 56140, 107549, 105642, 89182, 117382, 9661, 114212, 152688, 10236, 6963, 125359, 49017, 133271, 16476, 12541, 35616, 69159, 1629, 102356, 63487, 505, 73457, 56523, 22200, 74803, 30270, 49033, 133393, 86035, 97359, 122498, 21157, 84350, 48026, 9461, 141132, 73351, 10696, 7983, 34856, 23606, 84154, 25735, 134784, 45703, 157651, 71810, 110402, 124595, 123588, 147134, 125992, 31049, 15305, 36776, 105600, 121723, 55904, 45496, 10963, 47166, 85399, 128001, 63340, 129848, 131533, 104674, 99325, 95257, 156152, 30395, 97636, 106386, 100946, 14001, 45745, 753, 99600, 149282, 109319, 80945, 118334, 111487, 26680, 155730, 70683, 59995, 122799, 984, 46557, 151210, 93883, 148426, 78489, 13732, 27210, 84376, 71848, 13783, 121305, 46891, 2937, 138616, 142870, 35612, 61296, 122056, 125822, 38798, 63531, 85661, 81529, 93331, 62105, 39072, 67933, 57981, 30944, 88228, 123881, 87294, 74527, 133235, 134086, 113106, 147108, 140634, 29140, 19815, 6929, 45169, 58900, 106152, 81260, 147918, 93768, 87887, 30616, 74785, 6695, 109963, 9938, 83015, 51547, 127320, 142500, 97971, 156323, 63572, 40562, 36280, 71112, 38130, 138356, 8539, 94518, 89677, 137974, 121836, 117309, 20111, 72972, 73268, 121875, 142745, 115076, 112088, 70348, 54767, 149483, 23811, 89449, 101314, 124884, 107970, 143290, 117188, 119729, 128304, 112931, 151852, 26023, 63946, 33344, 80033, 54926, 42468, 146956, 40245, 135592, 20977, 157701, 117025, 35083, 65562, 156385, 83917, 108180, 104802, 135855, 151110, 131782, 44700, 40633, 115938, 60108, 47498, 60076, 43921, 154042, 25560, 44248, 126991, 144984, 13767, 41591, 2047, 69888, 55847, 78293, 139760, 59265, 127979, 134244, 128432, 124400, 155525, 88647, 8846, 28499, 133335, 51053, 94025, 72473, 51112, 3212, 142440, 73201, 88234, 83601, 35464, 23789, 38063, 147829, 8093, 36628, 43955, 141567, 147695, 53804, 43931, 29357, 46876, 4252, 114335, 147513, 405, 61565, 149481, 135967, 118871, 13805, 125308, 112287, 80697, 118850, 129199, 60492, 106727, 45575, 122697, 58331, 36851, 41852, 67626, 75328, 132359, 3792, 57300, 433, 15740, 104689, 33973, 132793, 1207, 84396, 70244, 84581, 32658, 94464, 80979, 62436, 104853, 87048, 54486, 140238, 136893, 46690, 16685, 13140, 128115, 33221, 32136, 114902, 117510, 97527, 51802, 85646, 78889, 11930, 50337, 152417, 104412, 33929, 109900, 88979, 58009, 29354, 146446, 43651, 26848, 99992, 111076, 19507, 130165, 43630, 4074, 50063, 93042, 158093, 62938, 95834, 15720, 145659, 117093, 4832, 52511, 77345, 18270, 31382, 98987, 15340, 20881, 15140, 142569, 156, 114962, 12898, 6169, 95454, 158667, 159114, 151953, 149767, 125302, 103964, 49215, 88082, 15050, 36809, 39340, 107802, 143586, 116283, 122514, 2970, 149706, 125110, 100975, 43460, 66824, 32459, 35623, 28136, 31567, 29337, 40626, 94297, 151866, 50463, 128220, 149411, 65529, 101254, 135697, 97407, 91716, 77469, 83076, 122216, 156693, 86928, 129956, 81424, 109206, 82731, 146872, 4686, 135318, 36262, 25627, 104828, 62749, 105288, 10304, 13913, 54040, 118680, 149182, 159087, 49747, 11759, 86276, 48579, 77143, 66556, 91705, 147409, 137255, 64260, 39622, 95893, 117189, 78163, 48447, 95036, 79507, 149142, 152510, 118128, 42771, 16793, 24913, 117164, 2220, 98192, 156749, 56704, 398, 113875, 138219, 125053, 64079, 71695, 33068, 47336, 123829, 86723, 10970, 117994, 144376, 12550, 4378, 46689, 137164, 25390, 49449, 13622, 69427, 3098, 120024, 91815, 132642, 32495, 135298, 157365, 75164, 101724, 36840, 82613, 3259, 71079, 10561, 45774, 48109, 55340, 71395, 84910, 148762, 7771, 48019, 7486, 45381, 92995, 113806, 139257, 15535, 22883, 140498, 75840, 117833, 147018, 78206, 5213, 74753, 73809, 114816, 58325, 122144, 18118, 3085, 154066, 23201, 143067, 30882, 145431, 34028, 87794, 86039, 33675, 154831, 137676, 128980, 77313, 25878, 134201, 114148, 149799, 113735, 40586, 131490, 30789, 3660, 96820, 36153, 51150, 144732, 117988, 59274, 94036, 6394, 3861, 4514, 28670, 40849, 27871, 104604, 125161, 143124, 154143, 110582, 37131, 130702, 113716, 102468, 125409, 100432, 141929, 41423, 110472, 26289, 130951, 158378, 18205, 112843, 31285, 49463, 141724, 135531, 101114, 37329, 88730, 27946, 125753, 154901, 127537, 39121, 24340, 157093, 36487, 76153, 114771, 132258, 70812, 88208, 42050, 40333, 118651, 134259, 6120, 41716, 120559, 18115, 35488, 149995, 123228, 47581, 36577, 110569, 72434, 83779, 46424, 110494, 43528, 59689, 84742, 106384, 149120, 28405, 30068, 143566, 15578, 74858, 35140, 92271, 159938, 5326, 139815, 44323, 4467, 69591, 19549, 128267, 29216, 87682, 36866, 142371, 62664, 76303, 122234, 76541, 140627, 87144, 110520, 88232, 35503, 1662, 113782, 101884, 26940, 148942, 58155, 1818, 140782, 99199, 154502, 92611, 129254, 90654, 128636, 90257, 125686, 66160, 94888, 35028, 63410, 96728, 106040, 146402, 16971, 101530, 64706, 113165, 116812, 52615, 88487, 92218, 53202, 63632, 99953, 63956, 107735, 79220, 44822, 76789, 103072, 100287, 7378, 11606, 62261, 34033, 43457, 145834, 74718, 69474, 79549, 67938, 84633, 81755, 21123, 17232, 81126, 23849, 16424, 96479, 123021, 111526, 122091, 65283, 72936, 49062, 1120, 114059, 98400, 48900, 27647, 87078, 117938, 92540, 153108, 125125, 148284, 53146, 4170, 65453, 49164, 159473, 91685, 126475, 104876, 148850, 119919, 75436, 110546, 119896, 146324, 84950, 124805, 24736, 87395, 82310, 71102, 59210, 5720, 128020, 83775, 54377, 142191, 97793, 5820, 17575, 90806, 123451, 132459, 157463, 28929, 109730, 118436, 151606, 41476, 156438, 146080, 158712, 57342, 56412, 57529, 84799, 46545, 92433, 66552, 111712, 144380, 157964, 81987, 5758, 91640, 115563, 131536, 46870, 108418, 143186, 118063, 6494, 55087, 105039, 152385, 121433, 13164, 79607, 89558, 112796, 127696, 80519, 94810, 111595, 19769, 20385, 84292, 145352, 143929, 145805, 54194, 40261, 31805, 95740, 83989, 68731, 72230, 156244, 74228, 32977, 71754, 158823, 60438, 47093, 4650, 132768, 14137, 72097, 142388, 110162, 34307, 33520, 143375, 19180, 17683, 116519, 89257, 81908, 28438, 127273, 74302, 84705, 105732, 18734, 10110, 110292, 102164, 49799, 64672, 67625, 41826, 47165, 96335, 70190, 67874, 106390, 157407, 78558, 36802, 92548, 8332, 157187, 75367, 50207, 157003, 120250, 130309, 118181, 96076, 140900, 87186, 70673, 72530, 65096, 105792, 104887, 135660, 59174, 31507, 42112, 17525, 58405, 23549, 154387, 66211, 12631, 30856, 122723, 115451, 109408, 121492, 49944, 78571, 90716, 147677, 158538, 8360, 92587, 57320, 94472, 38407, 19115, 54914, 96872, 122695, 148300, 33809, 39692, 43553, 50257, 84909, 13611, 116752, 107994, 136473, 18290, 100332, 108424, 1645, 135833, 74359, 134963, 37958, 37499, 56872, 92707, 58387, 7352, 93037, 72790, 41286, 88309, 158246, 85286, 157483, 50864, 151081, 130079, 159709, 57641, 116370, 53498, 45003, 46111, 47939, 23014, 149408, 68622, 58252, 119931, 37146, 79438, 104516, 61693, 62427, 145973, 103497, 75112, 81763, 142820, 136888, 101622, 71873, 16583, 117052, 53836, 40556, 108202, 85206, 24683, 118523, 111564, 70504, 121923, 86384, 35741, 159407, 134216, 107787, 141846, 85198, 132058, 4949, 68758, 39750, 73162, 134718, 19191, 39503, 80724, 123348, 22700, 81585, 81692, 1393, 24143, 141155, 157002, 3282, 15448, 32690, 20120, 114425, 48727, 150939, 150141, 124387, 34981, 132149, 128446, 118647, 67998, 43380, 5455, 5041, 134673, 118164, 154487, 8793, 47613, 17818, 49723, 49029, 157088, 44151, 7566, 51405, 144593, 140076, 133826, 27464, 119231, 4967, 97942, 55625, 115552, 14888, 45977, 103434, 65183, 112545, 119133, 26424, 11980, 106874, 103174, 27322, 83331, 103551, 57187, 140174, 121097, 30401, 150127, 144837, 141918, 102530, 104728, 115543, 90810, 119147, 100799, 19368, 31596, 149891, 137409, 77606, 58102, 44789, 45614, 32204, 68429, 67936, 58751, 66219, 552, 51254, 59349, 44299, 121146, 142917, 112599, 6044, 118219, 97543, 146235, 77234, 83517, 125637, 20787, 151809, 68362, 78135, 9150, 156067, 151556, 138745, 66262, 16683, 13695, 31774, 59509, 13441, 93273, 82916, 71833, 7341, 132597, 57198, 80478, 108270, 31439, 111118, 2524, 115219, 86389, 25622, 11330, 116639, 22491, 116115, 97129, 61620, 15230, 14647, 148, 46188, 333, 50178, 131130, 24156, 34522, 12556, 60726, 154366, 128387, 90033, 5770, 109166, 66942, 107904, 136943, 142554, 33946, 121304, 59420, 7897, 150632, 5124, 117332, 45044, 33370, 109578, 126488, 74703, 56508, 101306, 93782, 99735, 37765, 92113, 83533, 101776, 52100, 13000, 38258, 137878, 106686, 716, 102372, 91880, 145130, 56780, 30990, 99114, 141161, 23289, 64925, 117472, 137166, 76664, 107154, 61669, 130436, 53407, 85662, 18665, 82807, 100574, 86128, 121148, 116092, 63994, 5879, 100277, 144532, 70783, 11843, 10446, 2266, 136020, 103552, 143921, 37711, 126449, 60203, 46284, 142625, 10749, 29175, 99525, 26116, 15208, 127791, 122258, 64219, 38220, 85593, 116635, 59741, 105903, 7, 54123, 98585, 111092, 102353, 137533, 43093, 11651, 40497, 61208, 139242, 135320, 13661, 114724, 150452, 26923, 79987, 136199, 7934, 1402, 46611, 159663, 43750, 45078, 46415, 51245, 113829, 39515, 56926, 136228, 44316, 158447, 82121, 39479, 65498, 71668, 109483, 66188, 42809, 153647, 114704, 156947, 92931, 148812, 114164, 66912, 58721, 79801, 103331, 108512, 114521, 153760, 131718, 86433, 111971, 63076, 88637, 146777, 17563, 82914, 115822, 40951, 9618, 132922, 88394, 30183, 51178, 45110, 129529, 103755, 89605, 97139, 101625, 28423, 5247, 64996, 52817, 82443, 68949, 77863, 63855, 86933, 37443, 131219, 38044, 28537, 50983, 31052, 25526, 6286, 151032, 46900, 155597, 28700, 129804, 150806, 93118, 67917, 134041, 112226, 20635, 82274, 143102, 43298, 99156, 14696, 36801, 52480, 58408, 4511, 43812, 83246, 3462, 20074, 108426, 101100, 13346, 62688, 92596, 84722, 133674, 14154, 131097, 47556, 125875, 18079, 89427, 92381, 26548, 48980, 13032, 107654, 23611, 62332, 140513, 103067, 131543, 14886, 33898, 34682, 15353, 59713, 65874, 47393, 141494, 8935, 12025, 116632, 148557, 34569, 96863, 9089, 10846, 109069, 31693, 90715, 110778, 140776, 67231, 7519, 130221, 128522, 57652, 53122, 56217, 21211, 130515, 43375, 13845, 58520, 154814, 38832, 13607, 151580, 10117, 19703, 103436, 55926, 51370, 66851, 24167, 65974, 77700, 24477, 111376, 5114, 69511, 19130, 138302, 44023, 108988, 116960, 58899, 83604, 103676, 142557, 130742, 95856, 3980, 126728, 38247, 62363, 33755, 115333, 127096, 122828, 150460, 103311, 132114, 67403, 96018, 9075, 48161, 119016, 3540, 108209, 5160, 36114, 32284, 126594, 149601, 143323, 94447, 27102, 58318, 134811, 48236, 15594, 84872, 104051, 47279, 146662, 69137, 143161, 20851, 55269, 11096, 41481, 136579, 8123, 155099, 79558, 17038, 143754, 43150, 75323, 140696, 29006, 22184, 38117, 153485, 113872, 3584, 99188, 5252, 41977, 79354, 2098, 16687, 105798, 52737, 50358, 54702, 122698, 17882, 148040, 146566, 148228, 116307, 119465, 68416, 111377, 105607, 34246, 63080, 115410, 23780, 93048, 65490, 93951, 87210, 36579, 81250, 60246, 153512, 76861, 58039, 139442, 9856, 34477, 158308, 74615, 104640, 159576, 15280, 156236, 24656, 94299, 42478, 76508, 12867, 123994, 106278, 66966, 123173, 37399, 119294, 113550, 9209, 118608, 80595, 39599, 95691, 136627, 118406, 142626, 377, 65984, 121339, 103628, 156009, 34051, 58746, 33076, 73437, 115874, 117282, 106349, 41725, 7851, 156616, 156769, 16669, 2859, 47359, 41020, 42852, 110537, 88616, 149318, 38120, 43495, 68294, 90234, 153402, 466, 75921, 141221, 145585, 94654, 6540, 51079, 9104, 37803, 32624, 159309, 82964, 83329, 56001, 68139, 147244, 1853, 77698, 104162, 14054, 589, 102482, 59522, 47040, 43312, 15184, 86194, 82936, 18326, 148854, 28952, 107122, 31390, 142767, 71764, 47062, 133604, 119814, 125240, 16842, 140128, 79958, 47095, 6737, 143301, 149513, 124318, 14532, 147144, 11937, 68364, 153538, 126303, 34854, 11011, 48599, 49432, 33811, 135355, 71490, 147169, 46794, 70031, 33976, 72315, 31905, 92024, 68434, 52249, 79272, 50563, 59421, 35072, 60990, 46172, 99180, 124058, 126515, 41760, 29004, 30539, 148890, 36977, 117383, 83662, 110513, 67213, 132704, 28527, 37208, 137629, 10552, 22375, 44846, 149616, 93763, 117543, 106735, 52982, 107134, 120636, 148918, 98687, 20642, 8903, 20212, 115335, 78411, 110802, 137730, 150688, 40122, 132928, 141090, 36061, 32329, 96555, 157143, 93594, 89733, 128358, 48876, 103140, 67477, 83994, 122852, 71449, 74926, 53783, 88851, 61625, 15368, 48743, 102053, 81492, 57457, 154738, 154168, 145596, 126910, 55041, 54347, 2591, 23484, 42616, 135854, 94473, 128932, 72938, 17401, 10771, 79828, 3017, 142291, 53599, 93372, 24432, 89972, 127138, 78156, 16149, 31236, 81530, 25275, 16637, 129887, 92486, 74262, 113215, 64198, 51858, 48033, 136263, 23049, 6283, 154012, 24925, 47305, 88246, 37496, 108510, 47304, 81743, 28314, 8344, 57811, 90840, 75177, 42847, 34084, 98075, 44452, 8212, 54290, 58944, 28269, 58806, 58445, 74047, 159827, 153986, 43091, 73791, 12834, 96514, 138996, 145753, 13651, 58173, 129759, 114272, 110657, 67706, 86202, 110540, 46717, 124509, 135970, 113104, 30469, 29009, 62228, 66144, 48638, 6537, 99242, 103381, 8698, 98741, 107288, 525, 69218, 16219, 157528, 37812, 43005, 48708, 1842, 29091, 146000, 53969, 28382, 79689, 148011, 117014, 157023, 59984, 147877, 58843, 48867, 6641, 67061, 49261, 124539, 79910, 50082, 49323, 55422, 141584, 127901, 68865, 134518, 125718, 103831, 112953, 15475, 83229, 58863, 121891, 133040, 132745, 86375, 101981, 137236, 12604, 17859, 99790, 137500, 105048, 98557, 96804, 105843, 95185, 52412, 26775, 14643, 91154, 91163, 38320, 124379, 12575, 52946, 43509, 150535, 28826, 58543, 107018, 71425, 118968, 60013, 14168, 129681, 118617, 2286, 139622, 84939, 100924, 106332, 154182, 124637, 146227, 96600, 18733, 47966, 98216, 4049, 2103, 134408, 85833, 150878, 28719, 155316, 95824, 119207, 47746, 87800, 151070, 89116, 27702, 30464, 141072, 123247, 59622, 113890, 117744, 43656, 70892, 139644, 73667, 11132, 154241, 55330, 34561, 119867, 75656, 8867, 119852, 8926, 136724, 100595, 117228, 113469, 62608, 94275, 1495, 158698, 89579, 5017, 17717, 57564, 108428, 66619, 157878, 18977, 80917, 158927, 77275, 18602, 81640, 5804, 130554, 134790, 114155, 135480, 67425, 37676, 54796, 82137, 124962, 119591, 125919, 128721, 60275, 44428, 117475, 85233, 53598, 24621, 36230, 34606, 90965, 3780, 49870, 55260, 133435, 113041, 102161, 21508, 125429, 90350, 107434, 66283, 154352, 100601, 145573, 118915, 148896, 19919, 82003, 100870, 50687, 6817, 35184, 154555, 95495, 54773, 141076, 44653, 23443, 67823, 109632, 50374, 83689, 69967, 1305, 42608, 16528, 129314, 80102, 84157, 101575, 147111, 72661, 107425, 112851, 59105, 51774, 65364, 33528, 32052, 58936, 12410, 105150, 11318, 144201, 40700, 74935, 27052, 124501, 33829, 120421, 27991, 118314, 57599, 47612, 64984, 131004, 53275, 42077, 145143, 59699, 133267, 76406, 123919, 93257, 37252, 19138, 107316, 31830, 8104, 69040, 36295, 154494, 150165, 83935, 97076, 141614, 152191, 64255, 60562, 30258, 83312, 224, 66636, 97045, 159704, 40994, 39559, 62000, 25076, 117546, 63848, 38542, 67526, 57495, 9973, 120058, 87723, 154109, 107057, 1076, 140271, 139366, 113862, 108333, 4595, 96843, 79542, 133957, 126021, 68740, 20738, 31826, 23394, 78188, 135574, 131377, 10775, 126480, 23319, 8287, 133086, 27224, 47882, 107506, 14673, 50199, 154417, 102993, 10624, 129490, 67701, 76709, 99918, 75466, 13076, 44258, 67814, 20117, 56482, 48582, 39451, 157508, 135350, 129775, 45186, 60888, 112462, 94933, 64415, 15393, 5066, 44883, 18453, 130162, 69116, 71721, 53896, 116233, 44810, 95857, 2626, 21659, 11502, 102500, 37036, 93251, 14906, 16707, 156257, 94128, 55047, 7082, 48220, 40076, 145090, 101979, 159467, 40916, 130518, 27270, 116946, 6434, 29000, 98225, 18411, 131309, 24660, 141192, 106103, 3992, 20201, 92549, 85144, 27498, 114517, 88967, 57697, 152850, 130538, 106815, 15629, 18567, 120403, 6974, 30552, 72929, 78580, 85623, 26253, 42947, 73257, 98958, 144021, 102023, 41753, 141753, 152222, 83958, 154302, 22956, 41874, 128995, 72271, 105942, 56052, 55867, 108789, 84225, 17134, 127335, 81268, 94577, 82561, 31767, 134969, 32691, 107660, 28544, 110575, 126327, 16341, 5288, 70902, 18627, 107323, 74982, 28865, 131894, 157723, 111844, 130848, 144337, 146471, 44630, 86620, 34054, 79231, 8350, 138834, 17807, 117891, 53144, 136146, 145286, 139713, 148682, 20709, 113980, 141711, 119781, 4670, 59726, 6470, 2622, 145300, 118020, 39017, 92069, 127840, 152099, 2532, 151397, 59495, 119602, 2409, 128746, 40476, 150583, 92993, 125061, 74958, 108453, 151697, 77990, 2936, 142525, 136927, 100519, 153811, 7834, 154205, 63369, 139039, 95284, 133484, 122243, 18521, 125447, 151256, 113496, 115340, 138595, 20291, 143989, 10835, 68941, 29435, 46624, 22823, 123680, 40121, 26973, 157932, 55806, 101813, 64772, 118672, 148521, 111049, 31637, 148483, 141681, 64510, 127081, 129601, 107616, 56714, 61886, 68209, 90009, 5655, 115597, 37127, 61087, 142418, 19831, 100570, 48764, 101112, 127214, 68516, 27022, 1387, 122436, 147517, 157134, 106129, 12491, 59698, 137135, 36842, 62869, 115737, 63442, 114656, 115043, 120653, 103786, 45325, 87365, 96440, 118038, 151619, 145620, 106134, 150310, 72292, 11325, 67145, 64353, 152380, 84220, 78169, 81632, 137291, 67371, 18083, 127571, 139795, 95261, 84936, 52880, 155247, 81655, 46389, 16021, 4055, 56004, 32321, 113622, 71513, 82136, 4291, 8986, 39713, 78528, 66342, 156645, 136664, 65340, 20988, 107963, 48076, 146051, 90218, 65561, 110234, 147930, 155275, 36338, 62055, 136017, 112581, 38604, 142829, 108370, 95526, 14841, 89976, 145547, 125496, 35712, 117660, 14202, 132157, 135638, 127890, 20936, 21498, 96117, 85499, 107026, 38649, 55185, 135843, 22376, 118081, 106852, 45688, 135376, 60765, 119888, 88981, 87146, 59913, 82205, 154959, 43729, 6260, 138907, 113647, 53158, 45437, 101713, 83214, 74713, 140026, 53096, 77514, 121538, 83663, 82448, 99578, 50409, 117792, 103042, 69545, 113003, 138727, 39486, 48698, 89770, 102219, 66608, 113887, 87567, 150318, 84199, 50157, 143420, 69092, 50958, 75012, 108981, 69217, 34433, 74820, 50506, 10476, 20652, 13756, 146270, 31598, 77778, 126448, 149742, 20784, 120151, 99793, 101193, 67944, 159054, 80208, 143931, 36828, 1753, 46997, 83562, 95463, 129547, 1388, 33174, 44788, 36308, 115348, 15641, 110099, 19900, 64765, 40233, 42269, 7940, 101870, 111916, 83616, 29597, 97279, 48274, 148577, 1787, 141840, 115693, 147122, 35774, 87035, 151683, 11742, 154963, 66414, 111951, 159162, 76851, 879, 127512, 9884, 117000, 38283, 11972, 75171, 150107, 10267, 131024, 136895, 112870, 101926, 143802, 93247, 24934, 35471, 110927, 49060, 26318, 130024, 49453, 147889, 40872, 41135, 133006, 22052, 108648, 144370, 38311, 116173, 3000, 93164, 83241, 27354, 149751, 77539, 19868, 95033, 77112, 121802, 57446, 103314, 112948, 84207, 155178, 95546, 76151, 61462, 54292, 20252, 88513, 32991, 157096, 96541, 101215, 127509, 120331, 106664, 28512, 17538, 118465, 103671, 143077, 79971, 127560, 497, 79900, 74461, 8178, 82272, 120828, 17545, 137492, 138112, 153550, 158724, 50743, 59385, 77375, 84572, 140248, 131760, 53205, 115901, 20463, 127439, 73640, 82062, 129150, 15481, 138758, 90704, 100345, 59406, 116816, 137606, 89496, 133914, 69828, 114459, 86622, 133953, 29014, 114338, 7000, 128605, 566, 16930, 7760, 136890, 27642, 134042, 59715, 110123, 43467, 155681, 104236, 27913, 9762, 150637, 27967, 32499, 123799, 6346, 105374, 18948, 93753, 53385, 104403, 107176, 129674, 84285, 126201, 50634, 49599, 137269, 54202, 4988, 156329, 109130, 137496, 81952, 149169, 122453, 29308, 7325, 156445, 16657, 82119, 64226, 8437, 99564, 157054, 80142, 322, 5572, 29567, 95732, 48448, 3241, 152729, 55094, 19977, 139944, 31884, 40386, 73697, 22306, 70945, 70458, 83636, 83166, 82273, 100202, 21745, 156389, 46480, 129836, 154676, 89089, 78158, 127709, 93019, 111308, 68551, 73673, 48230, 51200, 70502, 135707, 106800, 133574, 148803, 149501, 128265, 71691, 146895, 121578, 52395, 6868, 130484, 24923, 91550, 145711, 157748, 108600, 53132, 84395, 68037, 140216, 88884, 31733, 124144, 83488, 120003, 103884, 157740, 133912, 107030, 84381, 81074, 3311, 144708, 99232, 118384, 82270, 94640, 41435, 116559, 107029, 31121, 141613, 111237, 51276, 121384, 90855, 24749, 145529, 69952, 57087, 137774, 8488, 114107, 140176, 7963, 87619, 138039, 22315, 134384, 76534, 67546, 18562, 41500, 78925, 79711, 73120, 91171, 80275, 41553, 114108, 67862, 138903, 11684, 79835, 19155, 49398, 23060, 101695, 96690, 122709, 15520, 38495, 96457, 77506, 155241, 93470, 120774, 139431, 67012, 67986, 100936, 134895, 31080, 145840, 75481, 51800, 22028, 135228, 76466, 97535, 56733, 70786, 113168, 153159, 41801, 13056, 114497, 64677, 125930, 16090, 145739, 135738, 139116, 5721, 78339, 2662, 136015, 110066, 76350, 7597, 144408, 87666, 68016, 117582, 21769, 124345, 36052, 115346, 113435, 84101, 80743, 29834, 61732, 9443, 79480, 152010, 132389, 74827, 4976, 18732, 116916, 146685, 103809, 100979, 13927, 143065, 31836, 72855, 55975, 59576, 138243, 108837, 87469, 131059, 69757, 83512, 86293, 21600, 78181, 25953, 118874, 143159, 119441, 9241, 22861, 32912, 73902, 125565, 49148, 112199, 58465, 100151, 68677, 142903, 131637, 106354, 53623, 140171, 117024, 24642, 87698, 142312, 82911, 15081, 108081, 89615, 4203, 100215, 142126, 133905, 127066, 83038, 101700, 42721, 60417, 143887, 38944, 92397, 152455, 145289, 136555, 79202, 42664, 100896, 121290, 75961, 83446, 10435, 92380, 100400, 139950, 22186, 21992, 12926, 22908, 74460, 51296, 17066, 130059, 152054, 154213, 55569, 122689, 143380, 25006, 34428, 69296, 150707, 81321, 139395, 74704, 61225, 18204, 108914, 145624, 13813, 51553, 85847, 96809, 122110, 118292, 15111, 133500, 48719, 1145, 23297, 100513, 102367, 93613, 157471, 97677, 55440, 90303, 148376, 110008, 96444, 61384, 73606, 127860, 131832, 93125, 49443, 155905, 153980, 26716, 126519, 27441, 10905, 2015, 118575, 49218, 15357, 15522, 113757, 114849, 43055, 65704, 122005, 23723, 41184, 30343, 146475, 54866, 118104, 128198, 86053, 56209, 651, 103925, 99655, 101035, 143506, 82090, 77610, 64377, 61531, 125932, 23664, 24051, 106037, 17386, 150224, 92501, 80044, 25094, 29539, 45116, 152244, 133340, 34658, 121639, 11439, 28065, 159512, 111809, 56934, 90447, 68049, 43008, 74370, 10091, 34130, 47847, 70735, 85592, 26955, 90686, 80437, 32783, 677, 18015, 142682, 38434, 50118, 138137, 51487, 86041, 72724, 94972, 113680, 65758, 155008, 81729, 73359, 86069, 51338, 54982, 86691, 118060, 144520, 26666, 58577, 158275, 152495, 6844, 17850, 48096, 77764, 122706, 82752, 76197, 79651, 83570, 140142, 36610, 67174, 147823, 63656, 1073, 109972, 85764, 43113, 25823, 155786, 129003, 45698, 95556, 10043, 82051, 41105, 140087, 101972, 140505, 98347, 62826, 62985, 97633, 112802, 40896, 43851, 84333, 47451, 112452, 14922, 63486, 89081, 133031, 33066, 140356, 128896, 140102, 72960, 46114, 90383, 67642, 14183, 56586, 127773, 50806, 72729, 10877, 99683, 36458, 726, 22910, 148657, 141479, 25455, 58008, 117738, 71291, 136013, 124816, 44891, 38946, 132380, 50349, 71083, 106188, 13526, 37901, 13811, 8066, 123879, 5940, 82582, 66341, 52662, 146513, 132759, 79447, 136225, 83389, 125444, 141093, 67679, 72248, 113753, 120248, 119066, 105560, 148625, 27431, 76362, 71213, 153039, 104011, 143590, 3535, 62514, 6364, 54373, 16417, 138738, 21357, 90922, 128744, 105437, 100123, 123097, 124680, 2675, 118727, 154023, 18578, 15849, 93467, 57698, 103000, 59323, 69810, 151525, 146628, 140124, 143991, 29186, 35568, 48323, 99026, 73264, 97399, 52728, 94805, 32026, 142944, 148308, 29516, 122531, 151884, 144715, 136468, 71113, 65070, 78654, 39070, 46921, 39787, 56463, 23987, 91240, 119646, 106462, 1514, 10983, 21660, 108166, 46544, 83102, 80037, 96701, 158078, 119143, 78880, 133480, 58744, 68194, 121048, 117977, 21186, 95846, 78498, 67806, 10277, 16065, 71473, 39649, 123234, 101124, 74765, 104104, 62651, 21911, 20551, 50424, 48704, 6580, 38787, 41188, 111386, 109119, 18596, 120529, 131283, 89935, 77905, 29168, 39893, 88091, 119308, 147784, 9366, 205, 124576, 36413, 23641, 74800, 43418, 40741, 68196, 43620, 128726, 48880, 59358, 3111, 141957, 2925, 69578, 54300, 78583, 22409, 128029, 102946, 60980, 126044, 97237, 156954, 105304, 100464, 54139, 82063, 53022, 24192, 149082, 112443, 32860, 153215, 71283, 63570, 9052, 26184, 67845, 124702, 119317, 34403, 142317, 43436, 11104, 91582, 16081, 80733, 85857, 41641, 93979, 62980, 141488, 41649, 25730, 75709, 57907, 54607, 72103, 14940, 67446, 42107, 110051, 47410, 75888, 57252, 24940, 130212, 87275, 14596, 53768, 53008, 112053, 49997, 8923, 56251, 124634, 20268, 104373, 144229, 118601, 103997, 53685, 83413, 140916, 124619, 38672, 28485, 40987, 114663, 22025, 71393, 140464, 89234, 149407, 96516, 95011, 95571, 122956, 60943, 113570, 43045, 82065, 63830, 66691, 16924, 1333, 38182, 30797, 152329, 153909, 1562, 93185, 93092, 18295, 74307, 59630, 85043, 152323, 151477, 116542, 25791, 69842, 140768, 60757, 43437, 55220, 128182, 116915, 69502, 25552, 68785, 75244, 117946, 21341, 8545, 1027, 69817, 82217, 114719, 54806, 31499, 103668, 100617, 116228, 114572, 69096, 155842, 641, 45412, 109317, 129109, 98555, 78942, 94821, 147372, 59957, 2511, 140974, 104479, 127904, 102460, 128868, 24984, 102186, 76560, 125032, 29035, 87426, 49415, 140730, 20071, 137996, 2878, 131583, 11196, 104187, 77395, 33890, 34248, 113656, 80983, 94355, 144986, 146659, 33289, 42157, 22067, 43913, 23754, 100667, 158244, 33606, 51007, 111180, 58053, 125756, 47728, 21574, 120935, 126574, 92692, 44515, 14621, 88049, 750, 13215, 151171, 138633, 37026, 117675, 51705, 102754, 63432, 125876, 88187, 132361, 42819, 88474, 9018, 42186, 30426, 130844, 69913, 105424, 52549, 1345, 50211, 76647, 26375, 108671, 135711, 1814, 16908, 80755, 33341, 4965, 30591, 95762, 85039, 115998, 70119, 67117, 106764, 112520, 75304, 30917, 34526, 70353, 103915, 78915, 77943, 9050, 82548, 112013, 122270, 159919, 37941, 59288, 85169, 139921, 123613, 126703, 112027, 55066, 22292, 68058, 68299, 122506, 107436, 81019, 54304, 131823, 16141, 61457, 91380, 16604, 152531, 39539, 84721, 106227, 104336, 43209, 48147, 30582, 51673, 137919, 108875, 99251, 4584, 130300, 149204, 151059, 137688, 128243, 94938, 149116, 57984, 118920, 112400, 46119, 74843, 63175, 129666, 141307, 24896, 89692, 152972, 42514, 108382, 91607, 117174, 15649, 92602, 105972, 46952, 155811, 124332, 82204, 22332, 59855, 17283, 10093, 75306, 62729, 22355, 135131, 54633, 117927, 85808, 107198, 146001, 54063, 53791, 57935, 122495, 111588, 130916, 119269, 99919, 138984, 30550, 94703, 144243, 145349, 101855, 46046, 74150, 83747, 2743, 45723, 109756, 156631, 36686, 127187, 88191, 116476, 130056, 15398, 24523, 43060, 77018, 86506, 48320, 112732, 22802, 139680, 90264, 77947, 21798, 99868, 47335, 137602, 62616, 117972, 156074, 136080, 85259, 140441, 98482, 138247, 83597, 80881, 32133, 58775, 154074, 61915, 19356, 65348, 79584, 129878, 52934, 44054, 14843, 79403, 48137, 159452, 37517, 56767, 75712, 93823, 100768, 27575, 13348, 114994, 93231, 53931, 2716, 81886, 132005, 84363, 134924, 51038, 142813, 94362, 118009, 31184, 18509, 74786, 122064, 80253, 34444, 33544, 98436, 96410, 7348, 78512, 74368, 44, 117881, 179, 136485, 73408, 54, 152883, 11067, 83805, 59115, 28376, 102908, 94004, 75642, 132327, 134401, 87975, 153171, 55004, 3405, 89471, 41662, 2250, 139240, 99995, 31668, 22206, 50525, 94971, 63893, 148043, 89034, 110042, 16377, 141771, 6843, 81408, 79070, 26073, 68898, 142123, 47466, 41372, 84992, 79997, 107041, 11060, 3739, 121478, 103682, 120013, 6783, 34781, 74338, 51031, 47762, 42756, 140467, 104175, 58921, 49394, 77159, 116515, 91795, 154747, 127396, 81734, 77129, 42271, 57365, 135974, 15482, 26516, 138728, 58745, 6588, 43239, 24748, 896, 32532, 150706, 103856, 114846, 48485, 114299, 92937, 10052, 30162, 86734, 77350, 142469, 135090, 115089, 58287, 5412, 9002, 62513, 151968, 87737, 42834, 92220, 128858, 51050, 104517, 105612, 62128, 114423, 58716, 44255, 81170, 145205, 105936, 40557, 28606, 26896, 125037, 156931, 95368, 86956, 54720, 113675, 121667, 31406, 69086, 107856, 152073, 98653, 67324, 125764, 70983, 121817, 39139, 32583, 103563, 137521, 112359, 52113, 119817, 49494, 45308, 125838, 26016, 127702, 59132, 83455, 148345, 116734, 62886, 147007, 111134, 100043, 158829, 136031, 59636, 151318, 104275, 67756, 143972, 95070, 17300, 17822, 17064, 19952, 132593, 82328, 12492, 45033, 44580, 50896, 71444, 21605, 41899, 89529, 152567, 55882, 102150, 109003, 100732, 28756, 139235, 35474, 139193, 30374, 101591, 91691, 37616, 90902, 76160, 33738, 149104, 86870, 132549, 88325, 30844, 7042, 29, 149876, 53222, 53062, 158605, 89287, 102387, 86600, 48749, 109111, 112712, 44710, 102046, 106723, 102707, 93301, 8754, 2744, 73178, 147564, 54199, 100769, 111361, 135794, 118116, 83485, 77832, 98289, 109523, 26949, 19878, 118433, 9229, 1694, 62100, 35581, 47987, 92083, 118728, 130492, 91732, 80749, 133272, 121092, 66432, 130280, 135127, 93405, 97468, 64833, 17832, 126454, 15675, 148439, 51599, 38766, 40939, 78278, 36347, 65598, 2133, 138014, 24718, 46453, 136794, 69375, 127437, 23941, 104419, 128050, 134373, 8473, 11152, 107178, 36075, 116167, 38843, 86050, 41615, 40911, 76549, 30497, 28534, 131271, 94350, 145035, 130380, 150952, 94635, 15009, 55203, 71867, 92166, 146757, 112548, 94517, 11362, 12324, 138721, 70954, 149362, 38360, 78371, 124723, 48729, 139666, 59036, 100489, 100937, 107390, 84218, 126648, 158389, 65880, 73059, 86961, 141961, 21998, 25926, 113234, 11523, 59122, 31293, 57709, 111234, 81637, 399, 107052, 17417, 61656, 128130, 14308, 76332, 63653, 41111, 44410, 65751, 17013, 64292, 29134, 50330, 33266, 113019, 102512, 125290, 96934, 73789, 59383, 44262, 133679, 69391, 4782, 155882, 117444, 144657, 156065, 50035, 132875, 76916, 19975, 5163, 106413, 154838, 67015, 8773, 64250, 123369, 145544, 70844, 118543, 51886, 48127, 22062, 159297, 59555, 80660, 102734, 66481, 157146, 18869, 92604, 156677, 24120, 40796, 99620, 6209, 28608, 60088, 21097, 42868, 35510, 113715, 66764, 114891, 125016, 151568, 131477, 70732, 35991, 8857, 6161, 5239, 151944, 33819, 86592, 100435, 50870, 3681, 54193, 38546, 110795, 26223, 10799, 76915, 94416, 149454, 85985, 35225, 49761, 44864, 52733, 68351, 57239, 26726, 123987, 96998, 48752, 143835, 7464, 54466, 121029, 153162, 42786, 120973, 12873, 76357, 69061, 17006, 153105, 87958, 66773, 116539, 61459, 151844, 153995, 88362, 52066, 156861, 107518, 147305, 4493, 72629, 96930, 19086, 24261, 76205, 137130, 31122, 51625, 13232, 100891, 108726, 151445, 119398, 103167, 132203, 139437, 101008, 91826, 56381, 28021, 36288, 116052, 157285, 125437, 64621, 12393, 49807, 130272, 88676, 12998, 20321, 108866, 116796, 55675, 36556, 34138, 136284, 114783, 55074, 135948, 97756, 117521, 151538, 4685, 125058, 125805, 40479, 155089, 158819, 112878, 533, 137713, 47026, 123483, 27520, 119979, 19105, 81439, 132456, 100354, 5689, 107543, 68116, 29856, 104108, 88185, 136556, 60894, 146494, 155598, 146844, 120049, 130145, 98234, 80926, 105810, 100821, 80220, 54522, 45942, 139454, 38127, 22400, 17376, 106707, 20031, 143678, 27338, 88161, 131597, 68716, 159152, 36905, 158514, 106105, 144282, 132004, 119637, 22534, 108551, 87004, 88299, 18031, 106298, 32381, 28457, 103462, 9548, 51795, 79080, 10420, 38669, 30862, 140901, 52525, 8526, 80958, 54743, 20137, 84236, 29815, 6720, 97776, 10489, 84226, 40600, 118151, 81596, 27502, 118024, 131722, 4137, 45226, 158067, 39347, 103777, 25720, 24828, 71284, 74178, 155672, 99803, 116078, 17650, 131238, 113271, 128836, 135043, 80017, 58853, 56939, 95628, 109948, 112750, 70138, 116685, 68310, 120207, 116235, 87472, 24673, 104697, 2840, 129217, 74522, 102381, 30579, 86893, 101244, 92129, 143695, 101850, 131954, 134098, 104629, 74675, 29770, 141240, 8415, 140195, 92005, 15997, 45877, 129914, 97219, 144578, 66501, 87417, 114336, 115227, 81504, 28446, 48885, 110800, 75751, 114547, 113179, 98238, 42009, 82541, 130605, 24929, 110338, 119805, 126002, 119741, 7073, 147752, 138883, 21448, 158573, 142568, 121165, 146143, 134206, 159557, 92237, 14014, 30275, 120263, 64133, 116057, 133230, 67886, 135349, 90463, 14563, 77068, 22387, 131465, 98397, 91233, 73811, 17573, 71288, 77044, 87224, 142434, 109288, 119456, 31777, 110246, 14528, 7732, 10413, 95937, 72437, 112126, 114549, 12445, 121350, 97297, 13569, 106013, 130347, 49393, 60622, 75161, 125460, 139812, 65354, 118014, 72243, 92638, 120995, 119275, 136754, 58584, 46136, 1820, 56363, 144063, 139507, 157902, 64175, 100764, 50477, 62277, 123732, 56356, 97680, 147608, 2017, 15973, 15490, 86156, 34391, 85075, 90637, 117159, 97832, 75673, 44229, 121426, 24525, 155387, 44237, 137632, 4250, 11799, 103869, 100349, 46024, 101732, 151295, 107370, 85063, 121929, 2952, 42183, 129121, 118200, 22638, 70491, 96951, 56675, 121881, 109143, 99636, 45887, 103232, 140660, 66468, 102744, 16627, 54052, 23538, 158011, 72534, 158675, 105857, 122044, 25724, 78810, 12419, 18840, 133211, 132060, 85392, 95838, 147865, 28570, 116944, 89458, 129192, 128386, 109476, 73055, 89803, 59697, 55294, 60719, 46060, 113464, 8809, 146602, 127810, 139961, 93801, 80316, 159756, 77771, 43404, 40034, 150097, 158415, 116749, 49869, 90084, 107903, 81665, 68617, 52496, 88767, 95901, 55960, 66548, 118008, 155030, 69234, 109249, 86871, 70326, 66043, 132244, 111523, 72639, 13773, 104315, 36088, 66495, 22423, 43702, 114054, 143240, 155076, 113794, 29443, 156531, 74989, 53569, 136873, 9889, 73859, 140980, 128013, 9193, 101469, 41525, 25613, 93549, 89373, 133343, 26346, 95889, 91339, 23810, 14285, 107064, 144108, 487, 48299, 8309, 117575, 69946, 111043, 117943, 127966, 35610, 35270, 118438, 16464, 86862, 103305, 148433, 105329, 133606, 62893, 36284, 111347, 63266, 21897, 146705, 100015, 113378, 94312, 100884, 21886, 98405, 68633, 17027, 59108, 75264, 109473, 101734, 139690, 2153, 72377, 43370, 128076, 42755, 36228, 17040, 84552, 14075, 65632, 105937, 69654, 5082, 46033, 44395, 131013, 34803, 13113, 28508, 124918, 109965, 96738, 48547, 147975, 34467, 26884, 98814, 18702, 24897, 32584, 134418, 98088, 108735, 159887, 3263, 151647, 27936, 108300, 34877, 94150, 133159, 73740, 29015, 80075, 51719, 127341, 42462, 124828, 65037, 156272, 80237, 105489, 57103, 84526, 117476, 21776, 156697, 76335, 39429, 83454, 139626, 59873, 21664, 120440, 152484, 12629, 124660, 65533, 108654, 54710, 67351, 57194, 86211, 18429, 81022, 137057, 138686, 20132, 103345, 12880, 39240, 79526, 50251, 89021, 34809, 41670, 28502, 36975, 141140, 129148, 129967, 31607, 75969, 12386, 45656, 132377, 81713, 108460, 112285, 38996, 30781, 102705, 113497, 109193, 118854, 113854, 135360, 50953, 124269, 118460, 117340, 134728, 20536, 42403, 88573, 126577, 116030, 60565, 102184, 79101, 9875, 102619, 21695, 132090, 148273, 86917, 108785, 9301, 33382, 159301, 149510, 61326, 154862, 59087, 1991, 39946, 49627, 142443, 15928, 143393, 67189, 125941, 135410, 92782, 86777, 125421, 92250, 115328, 40502, 71117, 11304, 119821, 8917, 39904, 103610, 34496, 146438, 35754, 31321, 100629, 80310, 87926, 88631, 119787, 133638, 13038, 113807, 68996, 134062, 138638, 34851, 139262, 56329, 9321, 62077, 148881, 116728, 36939, 18585, 150709, 8661, 128003, 82044, 145122, 6890, 93937, 119297, 105597, 40167, 98305, 133181, 115175, 51337, 155759, 56602, 7828, 102325, 113017, 126123, 24484, 57023, 112861, 84605, 131712, 97333, 43032, 113649, 125539, 89762, 142563, 108013, 101615, 41175, 22993, 95678, 88226, 692, 71717, 3683, 22606, 50702, 14957, 120494, 158458, 80249, 16849, 4691, 103159, 60174, 139880, 157696, 115266, 91036, 150492, 159532, 79460, 55866, 3421, 107259, 14359, 30417, 112524, 119160, 27705, 121079, 41687, 107333, 98510, 69626, 133864, 68243, 68059, 98330, 62013, 2134, 137373, 134488, 71589, 52914, 123138, 18507, 105360, 156893, 150255, 10708, 66852, 117469, 144210, 159510, 112602, 151100, 40559, 9804, 127999, 102787, 35936, 19739, 146785, 147380, 126698, 20451, 108285, 32932, 154769, 124170, 9130, 110769, 122201, 81101, 125848, 151504, 12020, 139995, 42909, 17174, 109356, 70887, 82846, 131492, 127531, 64160, 50215, 105622, 5199, 8425, 47138, 106033, 71090, 108590, 16919, 22835, 72256, 16917, 99779, 109778, 91440, 19251, 129165, 117955, 109934, 64812, 102321, 4576, 53470, 156075, 46879, 38680, 27132, 66819, 8064, 47051, 14908, 56593, 75403, 149892, 132488, 29773, 147621, 138420, 89753, 126173, 105832, 53350, 57333, 137826, 104261, 52500, 9167, 12549, 138133, 156920, 23127, 155965, 82921, 17185, 67052, 63136, 141740, 100510, 111410, 85451, 126117, 55955, 55553, 49537, 119257, 19403, 70713, 22723, 90189, 56218, 142716, 147594, 144748, 124803, 117573, 76052, 86106, 132079, 150509, 17476, 133966, 47285, 66724, 10231, 127238, 98797, 36599, 151463, 99507, 14591, 91236, 1324, 98223, 129680, 78014, 59029, 143140, 126150, 121376, 150554, 69435, 153200, 122182, 60221, 26570, 157913, 75353, 64179, 28450, 20831, 61451, 132162, 10286, 78867, 157684, 21063, 101978, 34863, 47145, 57803, 11397, 153414, 58143, 65512, 142855, 150313, 126151, 16045, 3146, 40353, 130866, 28584, 7533, 149487, 82183, 9803, 152896, 83571, 89210, 96045, 75217, 58712, 134877, 99482, 64116, 91353, 104680, 5119, 61995, 123744, 122362, 77372, 7720, 151861, 138662, 37531, 13068, 93398, 138544, 99784, 73694, 134998, 86185, 92225, 53852, 143399, 71389, 25300, 13659, 9307, 128769, 11032, 133532, 56150, 63310, 153899, 117981, 52208, 157600, 18745, 118459, 145033, 35580, 148098, 35309, 143434, 126111, 38795, 85980, 158193, 146006, 43591, 47385, 127062, 125615, 86648, 143784, 50265, 10824, 91924, 141373, 83879, 93295, 84028, 152357, 124361, 2817, 124189, 59831, 23711, 30124, 73521, 121103, 140093, 92346, 13871, 143331, 31088, 47185, 142107, 158563, 69728, 103929, 134926, 148755, 118067, 82537, 4893, 104132, 26383, 115482, 4703, 136123, 61438, 135815, 29798, 43427, 106756, 48295, 64407, 105658, 142900, 75069, 55455, 75379, 138790, 35518, 25901, 39145, 82489, 48268, 79947, 107292, 124638, 134692, 84628, 4640, 1700, 96256, 10823, 131331, 78820, 116114, 24189, 47714, 116247, 43398, 154815, 120635, 41741, 8795, 138195, 50006, 108458, 113452, 129488, 78881, 24241, 59632, 86410, 45477, 41163, 30633, 67287, 69415, 467, 15000, 137903, 128527, 18396, 71526, 56773, 32440, 79889, 27698, 112563, 139656, 98476, 61534, 123388, 63381, 33061, 50666, 143452, 93087, 78541, 57578, 121713, 64489, 5678, 98011, 156269, 111632, 113706, 98789, 96445, 127503, 48313, 48917, 110158, 25385, 80221, 97241, 141895, 114302, 151241, 153264, 75773, 6973, 7848, 81264, 60248, 108129, 147624, 143599, 117746, 146554, 39180, 153809, 141823, 37042, 30938, 16607, 147528, 44405, 116902, 131088, 86142, 127767, 152751, 66429, 130620, 95742, 78192, 38285, 89140, 60551, 45348, 155036, 66006, 93654, 130549, 19927, 103482, 56537, 25306, 133270, 51681, 53720, 42602, 116338, 139750, 81836, 43269, 112824, 25142, 117772, 31173, 42089, 119218, 102750, 96361, 70985, 97269, 84583, 38370, 148889, 109211, 155214, 71859, 491, 73416, 55404, 104532, 95694, 95221, 58974, 154957, 3865, 44108, 66734, 96614, 38371, 124713, 138773, 147348, 79187, 24978, 51164, 138791, 67907, 61011, 36536, 59501, 15874, 9981, 137385, 92168, 57033, 35898, 51154, 100150, 101995, 97516, 9456, 18180, 66231, 150766, 67205, 35825, 61624, 81179, 129583, 101542, 34927, 144774, 112518, 1117, 111782, 15359, 100445, 151622, 37210, 87063, 60031, 23215, 109474, 51911, 126057, 55125, 148359, 90008, 93038, 103932, 113181, 122466, 99501, 17079, 88431, 126133, 142746, 113509, 31003, 104536, 36562, 132184, 46336, 103979, 80878, 67234, 19936, 85711, 7598, 70117, 63363, 122996, 86071, 39557, 89424, 35404, 38651, 102434, 130330, 141542, 68304, 28128, 17703, 10293, 85303, 71582, 144433, 97852, 2369, 106270, 92793, 1836, 50508, 62733, 16475, 51220, 106508, 153604, 94085, 113037, 23833, 150587, 69974, 72957, 67457, 42524, 22170, 8772, 116075, 5737, 111246, 101777, 88434, 138844, 142330, 52826, 139650, 63313, 114046, 85489, 95776, 137456, 840, 127608, 150854, 74849, 44794, 65554, 66390, 128910, 144050, 3471, 133885, 130629, 69068, 100709, 43745, 123425, 27998, 71941, 40324, 44705, 153923, 46905, 19282, 74137, 1577, 63003, 36936, 64288, 17891, 54093, 30389, 54411, 118790, 159032, 154428, 71878, 66812, 5470, 134154, 110602, 52236, 124018, 159597, 103087, 19951, 5564, 69556, 37924, 158896, 76247, 25952, 40290, 46186, 44363, 138816, 110440])
([1], [63273, 80973, 136748, 66965, 140765, 997, 129097, 149105, 141294, 20602, 112625, 18537, 151108, 8094, 79216, 2043, 121519, 114642, 24787, 105973, 146532, 85199, 31650, 138583, 154441, 102677, 39940, 89980, 68869, 25775, 79386, 69276, 16551, 80759, 140016, 139563, 27025, 122756, 13728, 63342, 20432, 39616, 39461, 90030, 146692, 148155, 730, 37414, 109221, 15102, 108415, 145402, 29645, 104421, 39073, 89332, 33044, 89407, 4320, 115185, 158851, 76996, 35197, 94531, 144872, 99532, 25329, 60245, 11238, 18358, 121782, 106853, 72780, 132910, 111599, 2389, 107881, 50386, 101174, 115960, 109549, 1512, 117151, 1837, 15929, 18892, 123899, 74426, 63628, 52024, 12059, 78322, 124237, 104378, 28703, 147220, 50569, 148053, 82958, 149418, 90296, 61773, 26744, 148314, 31899, 70834, 19354, 158312, 120032, 143012, 80307, 1431, 2526, 72363, 6162, 68701, 150914, 59560, 22318, 20224, 60326, 26203, 126661, 37233, 57151, 156426, 86351, 89302, 89633, 128804, 55651, 16221, 5251, 127706, 126268, 122606, 110887, 120539, 76323, 15491, 131641, 83439, 134506, 114076, 2790, 155616, 49606, 96132, 49012, 20034, 156414, 121299, 123416, 81464, 97751, 141864, 141617, 126013, 149332, 39365, 137184, 61630, 77411, 111686, 153566, 126016, 154407, 7219, 123177, 92038, 77184, 105237, 17095, 89812, 20037, 45263, 38103, 19886, 24664, 142668, 3218, 52392, 70059, 141347, 149849, 88450, 16102, 64055, 121536, 117058, 125997, 12878, 48625, 12964, 139949, 86676, 117554, 69755, 57947, 59489, 80836, 91443, 71073, 76269, 15748, 120600, 8385, 8224, 3378, 33673, 8043, 111391, 37355, 41463, 156084, 153747, 80282, 67379, 31837, 89014, 24821, 135183, 135106, 16774, 46305, 95953, 156082, 15306, 74085, 50408, 17654, 83840, 19844, 5948, 35461, 45276, 33372, 83380, 1485, 127895, 82159, 93774, 18717, 44073, 101508, 123856, 87227, 3740, 17394, 143522, 101688, 26114, 26864, 78712, 152262, 147197, 71853, 19980, 73417, 28421, 107092, 36973, 29944, 54853, 54149, 12203, 85297, 32484, 124775, 155809, 148732, 126428, 87358, 30804, 150734, 27298, 27744, 129208, 109237, 86636, 65952, 87297, 94078, 106811, 71622, 7695, 83725, 107004, 147017, 81895, 14719, 87764, 97160, 156392, 86579, 108971, 82237, 30089, 43997, 104582, 159049, 58870, 60158, 131748, 38655, 90323, 88823, 11757, 141578, 98487, 35875, 58981, 93895, 42934, 138233, 108579, 78629, 81994, 101961, 46720, 120055, 123969, 92139, 6411, 92758, 61017, 97719, 127181, 152011, 21555, 28179, 116993, 77409, 79932, 33706, 71752, 42231, 99503, 147818, 122932, 115963, 14882, 74140, 91109, 49205, 72036, 3081, 67682, 11823, 117075, 84845, 102005, 144929, 114492, 78772, 62747, 154108, 138036, 116513, 77135, 17230, 40453, 99665, 138809, 147013, 92685, 41347, 142546, 58425, 37589, 80466, 85976, 134667, 29649, 123376, 114462, 93046, 102056, 26339, 90886, 18963, 46441, 48622, 64313, 72964, 97143, 121243, 81958, 86405, 128224, 70899, 26450, 105003, 143123, 58730, 12159, 38202, 3273, 80581, 86451, 43579, 149222, 18956, 98252, 64204, 36133, 46331, 50709, 138436, 7442, 148347, 87721, 28096, 134796, 99774, 44225, 44753, 12994, 87174, 57484, 19872, 89167, 19329, 70050, 120567, 158544, 88201, 40726, 34859, 118451, 136298, 25741, 144716, 77326, 59542, 128049, 122154, 70579, 125513, 136132, 99641, 77094, 89859, 98386, 21506, 159409, 41378, 134217, 68203, 121396, 120941, 137307, 123964, 46265, 1419, 124532, 49489, 155911, 34188, 35971, 114986, 54683, 59875, 123321, 112292, 22484, 109365, 28745, 15779, 48862, 121937, 123780, 71466, 7330, 105116, 21214, 91774, 73973, 44024, 59997, 109438, 103389, 154396, 12161, 121850, 142303, 15480, 24270, 56104, 10704, 110588, 119069, 67102, 137066, 12408, 132334, 126803, 65620, 15511, 123542, 46320, 50758, 70441, 149097, 51612, 100613, 99040, 133779, 112873, 56044, 9192, 44751, 57303, 40309, 64584, 109489, 81551, 97305, 87918, 42297, 60409, 86473, 16060, 136319, 50648, 70913, 65613, 65104, 147555, 154835, 105384, 66744, 94405, 53456, 129454, 108978, 142040, 82706, 16899, 493, 121535, 109442, 27578, 69347, 10936, 19515, 65154, 69325, 131881, 99848, 81454, 32104, 153727, 66455, 35171, 148882, 72720, 133004, 27880, 145347, 21902, 63404, 66185, 127192, 148459, 121170, 100194, 103407, 136590, 128741, 49225, 46866, 118808, 48169, 92872, 36995, 143166, 43791, 103136, 119346, 66904, 62350, 6191, 71811, 862, 139547, 129237, 108055, 42791, 93036, 101915, 10513, 17653, 93084, 81816, 13319, 128814, 73800, 27444, 157908, 78076, 140688, 138245, 119732, 52633, 11685, 93640, 156241, 35768, 153448, 91088, 33038, 147132, 47180, 79482, 20763, 26079, 88850, 147315, 131938, 58236, 124489, 82396, 132476, 73890, 69826, 51145, 44817, 103046, 30228, 18751, 29602, 145542, 10654, 97757, 20595, 110030, 47355, 29988, 13809, 141939, 7402, 53220, 98188, 150221, 158262, 143942, 130381, 23883, 96153, 116223, 84074, 155212, 21191, 12055, 42979, 61155, 100611, 131619, 20563, 2607, 65111, 130584, 50592, 90550, 126982, 4409, 101783, 8117, 65793, 54626, 141928, 32178, 92198, 93392, 81047, 16938, 4159, 45313, 106149, 16099, 65458, 106899, 42492, 131031])
([2], [95848, 155294, 62093, 46862, 16955, 123409, 112140, 105224, 43798, 152825, 2954, 30336, 122239, 134037, 110478, 145319, 139928, 70303, 121847, 21202, 117960, 106497, 21103, 103994, 105610, 17409, 79591, 149859, 127612, 93488, 5336, 93515, 100779, 44206, 52579, 49681, 96947, 19414, 110345, 8395, 66774, 118130, 78611, 122981, 129578, 116549, 84656, 111442, 87656, 75198, 86538, 60392, 139098, 80533, 25833, 106728, 127332, 30047, 14396, 20863, 32737, 155799, 125564, 30330, 117934, 149591, 116149, 134070, 153251, 114083, 129033, 16675, 115550, 60406, 77788, 109183, 151838, 97701, 9897, 51830, 54620, 136424, 58050, 66886, 145121, 52326, 1065, 84258, 156660, 95523, 92661, 128615, 37873, 64857, 348, 147263, 79735, 145427, 89341, 73000, 73931, 134111, 25787, 45973, 144722, 157380, 154290, 155025, 149952, 10216, 106367, 76182, 98290, 5346, 149743, 125925, 62530, 63297, 94485, 157596, 47459, 146360, 148332, 72861, 10289, 50499, 21985, 2094, 102786, 53309, 29381, 86614, 105235, 68626, 63129, 131267, 28027, 116775, 14624, 100165, 142206, 58431, 88392, 73478, 34142, 98517, 45970, 119433, 94967, 64634, 90104, 63942, 28733, 57385, 114553, 24944, 156196, 139323, 27710, 87686, 84533, 103647, 34274, 18690, 127499, 97617, 50989, 79113, 144304, 156997, 129709, 151575, 73025, 115319, 86085, 124761, 48912, 52999, 1854, 113030, 63970, 74759, 114893, 141716, 9796, 29239, 114623, 86207, 134462, 111471, 117297, 98027, 23133, 53060, 146546, 138707, 29299, 154328, 17762, 34451, 125557, 122589, 98314, 10847, 12091, 148417, 79805, 76109, 115244, 157738, 22953, 31986, 30278, 94235, 116806, 64505, 71239, 47660, 150915, 133812, 23415, 150020, 147712, 45380, 156209, 141577, 130841, 132608, 99491, 32725, 1805, 87695, 136352, 14924, 154846, 148454, 83968, 15915, 84162, 6526, 36696, 83252, 2225, 37507, 101171, 154385, 28507, 88608, 84737, 15090, 8495, 85506, 60619, 39118, 24272, 95579, 511, 115093, 39123, 109203, 125865, 117283, 80639, 86097, 117887, 154844, 109419, 96103, 125489, 47855, 118013, 109478, 17342, 86582, 109611, 77872, 84111, 67731, 41703, 83072, 120084, 104490, 20715, 91596, 117930, 29791, 123433, 119350, 14505, 48968, 68646, 130562, 136200, 29309, 148951, 118025, 121992, 54653, 109746, 110372, 139296, 72844, 139568, 95380, 17928, 8860, 81036, 109236, 150370, 77757, 91745, 135507, 100904, 113376, 124351, 94193, 129927, 121529, 33216, 110805, 90570, 90037, 116562, 33036, 70911, 87778, 32913, 136377, 28327, 109808, 86311, 89913, 84272, 127147, 118304, 104087, 24077, 100797, 114806, 121088, 70149, 58587, 50637, 40539, 150620, 74797, 125248, 81718, 106753, 118700, 20598, 120472, 122241, 139351, 91237, 55328, 89435, 32770, 43125, 129870, 27130, 133610, 72347, 86829, 84209, 24492, 116684, 155341, 114881, 23959, 156721, 144075, 122861, 98228, 24850, 106665, 16109, 119479, 96527, 115492, 56065, 114683, 137832, 143106, 79764, 87772, 32949, 46909, 112636, 70042, 156810, 123792, 108963, 109407, 16408, 152489, 97759, 88928, 141829, 20167, 65672, 6596, 82682, 52899, 139858, 64684, 44651, 119585, 138287, 157020, 121799, 111250, 85448, 11712, 110145, 14550, 142973, 30354, 154548, 128309, 42772, 34240, 147998, 113461, 38643, 87822, 22229, 22389, 33082, 19100, 46498, 159816, 104395, 157103, 148649, 68368, 125293, 90012, 25917, 10240, 62710, 70420, 56870, 73295, 142452, 149543, 158641, 155233, 75316, 118529, 103061, 38286, 67705, 16615, 11299, 120571, 150603, 20164, 32081, 100806, 132864, 141598, 145262, 140484, 65459, 80647, 41459, 105711, 18197, 24060, 145050, 100962, 139238, 65442, 105429, 14627, 45304, 151508, 65094, 27731, 139882, 96877, 159774, 1954, 51104, 131458, 419, 30157, 117999, 103527, 67349, 127294, 25282, 116088, 150149, 8150, 102350, 104735, 139792, 67480, 107599, 68320, 57317, 102208, 116099, 28168, 41908, 119187, 59639, 17214, 105983, 6462, 103759, 20616, 3608, 98760, 153486, 95611, 115865, 31688, 131421, 88386, 17026, 6238, 4142, 53360, 88314, 4450, 111579, 60051, 49366, 50728, 90949, 157326, 154644, 132705, 52973, 36092, 113441, 44120, 58147, 132576, 154742, 85982, 57175, 123500, 86633, 88021, 75268, 2021, 73179, 37588, 63836, 81869, 97566, 19556, 37801, 158693, 58357, 65033, 135464, 144008, 87125, 121074, 73293, 119371, 121591, 29130, 62279, 96573, 12993, 30238, 18780, 18901, 104253, 26638, 61090, 71906, 106219])
([3], [102578, 83026, 110988, 42054, 44137, 134959, 144125, 45669, 153128, 158208, 21927, 153869, 92714, 141385, 103581, 65402, 35729, 71206, 85869, 55991, 80834, 95286, 54012, 51850, 71209, 147682, 144707, 58513, 132078, 12092, 151058, 118742, 142396, 94391, 139681, 138075, 90270, 59011, 83142, 92291, 134967, 4948, 34162, 34982, 97224, 155300, 92370, 153275, 11812, 39141, 39470, 119801, 158622, 2725, 33614, 7965, 156848, 71455, 154343, 37700, 66929, 96945, 31312, 53476, 82402, 123308, 38023, 109354, 115672, 61540, 115060, 138250, 114746, 101411, 72019, 45042, 21427, 133266, 51643, 39229, 72400, 142245, 86509, 152616, 126601, 108450, 110592, 121830, 62677, 112825, 44478, 92315, 59731, 148694, 124273, 27366, 44383, 117430, 63470, 54235, 46901, 89267, 85644, 159589, 89108, 99478, 130566, 130217, 59941, 49812, 159018, 156709, 132525, 19038, 47775, 133522, 99428, 32989, 92026, 8327, 38416, 45486, 59331, 152353, 117493, 37758, 20762, 118356, 46190, 53683, 85069, 69930, 5039, 16105, 40197, 124693, 144378, 150650, 92696, 859, 41417, 106984, 132851, 1175, 3844, 9159, 35164, 11313, 74440, 6178, 119689, 110720, 146491, 153375, 148500, 96489, 19443, 24301, 140720, 82947, 16591, 60449, 46797, 63885, 6664, 99048, 104622, 120788, 39855, 87100, 39722, 27400, 141653, 115870, 53903, 127265, 13430, 104706, 44688, 34746, 37905, 135838, 129851, 135760, 52617, 85588, 17366, 126308, 135962, 7699, 78779, 124348, 118271, 21845, 19416, 10321, 67696, 21210, 117755, 156870, 15866, 159436, 154137, 79032, 10105, 3596, 65382, 71736, 124766, 66977, 11457, 125079, 117349, 126763, 152216, 127598, 134370, 56461, 154539, 149468, 19616, 4616, 46699, 127674, 40941, 86740, 129811, 50854, 68611, 130643, 87707, 148949, 147328, 40099, 145236, 76729, 18763, 74642, 25703, 51988, 2142, 130150, 67805, 28680, 115601, 24503, 142614, 922, 106948, 49683, 21409, 14822, 146788, 118440, 50731, 129501, 156932, 79338, 131724, 71581, 150474, 60972, 805, 91743, 96417, 103952, 144878, 16498, 113605, 61096, 55856, 98659, 93283, 48822, 140565, 127711, 149232, 73088, 54357, 29233, 158475, 14577, 129479, 31684, 25021, 58918, 104553, 135779, 74217, 47268, 71474, 5020, 46660, 99043, 50024, 88006, 23628, 127304, 142421, 83634, 87763, 56522, 23498, 119084, 80216, 7621, 97858, 1673, 47177, 26377, 140514, 97505, 135619, 43496, 1929, 80503, 126885, 8228, 126023, 68761, 115322, 159376, 39419, 104274, 89135, 81680, 13425, 107722, 106601, 123839, 52399, 156032, 78071, 124606, 24852, 41946, 138642, 153043, 619, 116678, 79856, 76478, 53631, 48570, 48254, 137669, 84135, 119597, 81487, 89024, 140382, 130186, 22619, 132255, 79001, 104202, 44960, 6917, 24915, 159607, 73173, 50781, 121168, 53159, 101429, 77077, 53131, 124172, 45706, 111964, 119533, 157028, 4786, 41932, 148635, 159200, 11081, 4484, 103014, 118299, 42371, 156667, 141425, 147073, 108223, 30610, 88421, 148797, 70069, 59248, 76822, 24008, 31766, 63667, 45853, 63166, 154724, 143108, 68354, 77356, 74901, 73031, 128586, 65831, 31524, 112866, 80223, 147215, 84601, 35099, 132817, 13422, 154763, 110026, 90354, 37404, 120041, 122273, 6679, 126410, 104659, 22627, 119540, 149708, 87758, 67902, 77650, 104940, 105828, 77457, 91664, 139476, 50258, 136224, 128886, 114802, 84465, 14000, 49875, 66651, 130013, 6336, 14414, 124042, 118745, 28044, 115881, 87951, 65995, 79192, 64731, 134130, 45168, 89149, 63288, 66777, 144429, 106522, 10583, 26895, 155237, 21334, 85240, 91124, 42046, 87623, 107146, 104546, 38305, 126766, 86016, 23568, 140330, 111093, 158358, 39245, 90370, 67781, 63523, 25496, 79344, 150473, 126827, 137147, 84986, 4942, 112708, 21331, 50206, 136832, 69507, 73727, 73150, 124096, 17954, 136977, 102898, 40094, 43679, 74979, 105994, 12329, 81999, 21997, 13776, 67007, 144379, 37936, 4835, 65222, 46871, 79861, 149499, 110106, 157874, 39548, 129198, 64867, 136686, 19207, 106825, 134297, 138793, 46987])
([4], [70852, 140242, 85338, 95105, 148636, 91379, 8483, 123657, 74588, 72121, 30843, 45754, 141042, 102906, 3069, 96569, 46618, 94877, 69463, 151875, 114546, 140358, 130675, 83414, 93893, 66276, 122898, 18084, 78140, 135577, 14758, 54270, 150759, 130400, 94962, 87775, 32838, 36400, 151905, 76065, 52590, 159538, 16373, 85122, 32168, 15729, 27718, 76099, 22971, 97124, 78923, 86644, 11205, 48853, 36629, 79248, 72150, 13887, 59238, 108943, 76975, 125030, 48181, 35804, 101556, 108032, 75287, 19883, 27883, 13954, 11976, 69048, 67428, 6456, 25368, 66664, 99860, 47819, 76298, 154374, 3135, 61033, 139114, 90120, 28188, 124480, 89249, 56452, 122135, 79686, 11118, 119417, 11938, 79417, 5635, 135726, 139938, 10622, 44502, 154851, 138933, 638, 59429, 118853, 88099, 38914, 141913, 5983, 67597, 85795, 66382, 8389, 37509, 78392, 79095, 130273, 6545, 62378, 62485, 104824, 89413, 15064, 147811, 45902, 59673, 46108, 130404, 55514, 105465, 7272, 141243, 32727, 120006, 79970, 1610, 2596, 22531, 159723, 158569, 95414, 30419, 137339, 56713, 69561, 118769, 140034, 122673, 134415, 38514, 74096, 112644, 28960, 54892, 116270, 19636, 127555, 11021, 88700, 102319, 20558, 137100, 107512, 18855, 50868, 66162, 6878, 121446, 119558, 42704, 73458, 126394, 72752, 78984, 112829, 73112, 82333, 136241, 53979, 5734, 156348, 133025, 5292, 61867, 150666, 108071, 90436, 147541, 37028, 77760, 52657, 23691, 13812, 7862, 150170, 81412, 50229, 145139, 55759, 13004, 99, 5602, 80391, 74281, 73499, 81809, 90233, 157229, 44048, 64165, 95993, 104730, 149180, 113808, 10806, 140072, 111136, 46945, 81899, 116779, 136472, 98730, 34123, 137247, 46360, 120293, 58959, 105342, 112328, 27651, 35478, 23421, 24823, 63231, 143125, 34725, 8454, 108920, 126801, 126580, 65155, 135455, 13034, 108918, 47100, 135284, 50594, 87859, 9037, 138294, 66534, 42684, 33389, 45269, 94201, 45237, 158817, 88054, 72310, 24500, 53680, 46245, 61458, 138965, 46317, 13892, 56010, 35550, 110954, 69473, 151962, 32748, 21283, 98791, 3477, 62280, 133718, 64269, 147229, 40436, 24661, 21865, 29672, 17429, 53388, 55958, 104164, 94928, 38586, 153866, 117657, 135887, 66598, 10496, 59183, 143485, 46419, 82887, 6328, 47999, 17694, 123298, 150164, 136570, 106659, 44138, 74254, 92716, 131725, 136439, 106094, 105929, 25898, 61192, 39037, 142318, 538, 83821, 124893, 102115, 102037, 43576, 72246, 44666, 31741, 113886, 54457, 68237, 2495, 427, 88111, 153892, 48789, 47504, 14106, 25055, 26988, 26970, 113587, 70797, 63930, 145640, 118824, 104036, 129909, 134345, 51875, 114220, 26595, 135024, 97048, 33560, 55461, 104851, 53424, 122512, 152556, 18513, 70424, 103079, 75920, 118222, 118429, 71230, 98803, 6598, 13989, 142307, 16128, 90468, 109279, 115011, 38106, 83977, 143806, 13593, 42840, 77199, 42136, 113659, 130853, 15767, 91707, 66779, 5127, 64872, 131253, 128593, 3099, 141346, 84062, 137954, 91217, 108537, 143321, 155318, 131570, 5369, 104463, 20431, 60169, 83016, 93427, 36787, 43477, 89725, 156482, 118027, 67397, 155480, 46170, 6792, 49774, 15473, 159931, 80479, 353, 72756, 86193, 8663, 118798, 61633, 106361, 98956, 92894, 101553, 74888, 14659, 112987, 86518, 28784, 101847, 30051, 157989, 94776, 44038, 121522, 147120, 138008, 43653, 157104, 106254, 69213, 129388, 32537, 85444, 149760, 134641, 159536, 141720, 119354, 94356, 145523, 91523, 97604, 142433, 8181, 6166, 137452, 146151, 52166, 8710, 51114, 135278, 125879, 95138, 39490, 107588, 63430, 96225, 96625, 22212, 139708, 105354, 98922, 146477, 85467, 110438, 22117, 139252, 16937, 28233, 86467, 2124, 120051, 134044, 144745, 69758, 67254, 150493, 53357, 130910, 38133, 47445, 31019, 20470, 45123, 54371, 150208, 70198, 45584, 34548, 102740, 110757, 4332, 131272, 107227, 125798, 14108, 72655, 31310, 115670, 42902, 41439, 102880, 88346, 84974, 156695, 143062, 9789, 130941, 143093, 52306, 118313, 134624, 113665, 42437, 141991])
([5], [96596, 131240, 147481, 52834, 75497, 113454, 22520, 159693, 42121, 157133, 17284, 14349, 93910, 134476, 66151, 22736, 3362, 28741, 110159, 1896, 34152, 154768, 22439, 115408, 64943, 128692, 116507, 143080, 80269, 112677, 100579, 129818, 113285, 105759, 38800, 891, 122423, 87726, 50808, 131839, 138416, 143312, 87730, 82280, 17795, 76022, 49637, 109081, 18756, 33678, 109486, 63346, 108266, 37027, 21376, 150278, 2588, 42242, 66938, 156502, 6160, 101562, 30100, 96415, 138964, 6814, 100851, 138130, 4229, 2515, 47752, 3756, 146666, 49957, 78376, 31686, 133731, 104693, 47061, 57695, 125490, 29196, 53845, 74438, 142984, 133176, 152178, 69657, 5454, 31217, 59785, 155540, 59824, 26929, 42229, 113323, 19393, 116318, 118600, 64390, 42882, 89978, 84466, 4807, 108892, 90166, 13103, 12509, 104055, 17682, 144854, 153942, 52016, 63538, 2251, 68145, 33576, 129554, 11109, 50397, 31585, 83849, 70318, 103774, 88361, 36430, 42254, 115816, 65493, 92958, 130058, 77272, 157677, 61315, 80928, 124989, 22798, 93588, 31511, 154884, 158135, 92769, 127576, 146053, 25024, 20737, 39845, 44839, 33381, 3473, 37445, 93311, 137569, 41961, 108244, 36946, 100537, 38056, 86932, 41583, 31972, 101086, 30437, 3577, 91753, 40221, 44828, 157202, 47879, 44477, 40779, 52832, 144150, 157099, 30506, 47821, 141120, 144058, 159102, 27881, 157853, 50716, 153184, 153874, 111216, 119752, 93538, 60342, 70595, 33052, 105423, 26913, 105244, 20264, 53877, 32513, 105746, 15771, 60489, 18922, 8271, 71886, 101275, 54413, 81663, 15161, 107670, 151902, 153032, 87350, 40035, 83377, 67232, 67594, 61204, 122804, 80621, 134175, 111828, 18814, 123450, 101173, 84104, 137351, 58602, 22334, 89278, 22759, 135975, 22033, 40150, 125726, 71172, 120380, 156711, 73943, 43634, 142878, 40672, 117733, 61461, 47832, 150288, 20888, 30480, 128259, 94975, 142038, 4631, 56434, 153491, 19498, 111409, 99917, 74798, 73738, 40720, 77648, 7474, 99410, 78180, 56613, 90278, 110687, 26423, 113566, 141052, 56214, 65733, 31696, 132893, 67913, 7055, 141702, 40529, 151288, 59215, 128668, 5700, 108721, 18200, 48537, 50675, 92219, 82022, 3276, 57231, 22080, 42901, 91142, 2827, 127150, 177, 137160, 82904, 3258, 59664, 53046, 2030, 37864, 93179, 88620, 78938, 4281, 77420, 3175, 149682, 114476, 78321, 24578, 87805, 100156, 20493, 104758, 155590, 99504, 111016, 61497, 6318, 118862, 133769, 135362, 117796, 6910, 90571, 61576, 44159, 25254, 60628, 32175, 143816, 138865, 65819, 104791, 153893, 8278, 99791, 127842, 113064, 158659, 13963, 36025, 33724, 34429, 93761, 154445, 9432, 61159, 150593, 143078, 1893, 155117, 125894, 141142, 134464, 141935, 75603, 60230, 102814, 51603, 88925, 111776, 50131, 74723, 11532, 85125, 113682, 133712, 7924, 24420, 61875, 19271, 39105, 78985, 137448, 148309, 59074, 157522, 134722, 56784, 146292, 6116, 84436, 77062, 31806, 21748, 25294, 73122, 3837, 105253, 43551, 156079, 23907, 80645, 119721, 155751, 152570, 91412, 4795, 84882, 115856, 142144, 60901, 106166, 120890, 90809, 99856, 97896, 105143, 14332, 33612, 146449, 62027, 818, 102358, 21518, 105163, 38276, 124906, 58204, 86324, 67762, 3023, 5296, 8365, 154139, 60932, 33051, 157537, 151971, 156791, 107835, 147322, 147385, 129722, 152287, 128114, 13524, 40688, 82461, 6100, 149781, 24457, 13274, 155213, 103564, 59032, 40816, 61653, 3002, 95271, 96670, 53679, 42965, 130581, 6341, 58375, 115834, 10393, 144550, 24518, 42515, 35420, 36278, 120974, 136693, 64673, 159181, 67421, 4157, 57929, 23915, 60180, 77789, 1266, 110401, 68192, 13291, 118057, 77413, 29512, 113610, 97964, 141505, 105654, 141763, 44220, 25814, 42361, 27545, 92479, 79621, 91519, 127010, 133937, 125307, 51778, 40295, 95122, 105944, 121118, 78517, 148351, 149749, 28655, 10832, 130611, 157436, 111367, 40934, 115528, 146254, 15589, 234, 117549, 4063, 96521, 124812, 133383, 70767, 123452])
([6], [49085, 92241, 11888, 58914, 67310, 114169, 103811, 50986, 106802, 74175, 50938, 145221, 28403, 104382, 142260, 107876, 70256, 8801, 38131, 136894, 67411, 79414, 2280, 84557, 29620, 105026, 97599, 58350, 102330, 10756, 11841, 31357, 137367, 3630, 77496, 89943, 129348, 90035, 88280, 140030, 81465, 72118, 54940, 128823, 14125, 33864, 158342, 75926, 81208, 144820, 106460, 47747, 96381, 121052, 85962, 76479, 50283, 107775, 80525, 78828, 112808, 82305, 45860, 156040, 137146, 118236, 24781, 54545, 37448, 78670, 144295, 137649, 34716, 46486, 10309, 151657, 140178, 155504, 57758, 152902, 127629, 4424, 151579, 87008, 156227, 137979, 60668, 36923, 53087, 84355, 140310, 93865, 21814, 43875, 71012, 117413, 62351, 91777, 152722, 60077, 27549, 91433, 79, 4871, 128286, 6259, 39827, 104291, 155027, 49740, 117893, 111856, 44801, 136054, 158991, 148276, 34483, 62049, 42575, 26936, 96687, 127685, 134132, 140443, 78708, 147008, 116039, 151001, 43817, 9160, 137779, 92012, 149687, 126382, 113839, 82673, 30600, 144680, 70860, 59676, 73598, 21177, 144115, 3357, 150762, 4249, 131408, 12773, 103480, 79941, 152267, 136588, 68366, 47092, 51644, 139952, 41042, 112218, 108895, 94730, 154301, 34753, 125089, 70541, 31589, 7775, 142664, 36154, 29685, 11494, 142718, 120333, 93095, 111648, 93404, 35094, 55504, 91698, 60695, 131658, 61804, 94562, 134843, 44183, 41232, 59908, 93921, 68575, 32161, 68601, 105528, 63414, 146543, 96786, 98205, 19905, 70708, 32163, 113475, 119974, 137857, 68885, 23461, 152520, 130402, 72276, 128782, 4038, 140359, 159303, 126435, 54917, 93099, 18554, 64396, 54553, 73297, 184, 153543, 37510, 50835, 82820, 21731, 96694, 126125, 29686, 116891, 155360, 18080, 143579, 158653, 136231, 59889, 74563, 157352, 115208, 53711, 133229, 37797, 120481, 136348, 56700, 105389, 139958, 30730, 106512, 85092, 140052, 32223, 94032, 28708, 98892, 105161, 105018, 55016, 140701, 1004, 153748, 98159, 96605, 159713, 65326, 108904, 74078, 139189, 154244, 30054, 44205, 65312, 142946, 8112, 85998, 88407, 85743, 154453, 56519, 85067, 97467, 106750, 123310, 118248, 37887, 4735, 84123, 129612, 138175, 38414, 3903, 7548, 138040, 59532, 115694, 123377, 18491, 77963, 107222, 13787, 7687, 9874, 37228, 63195, 7648, 32914, 118469, 114052, 131707, 159380, 80403, 132833, 35987, 139639, 50047, 50475, 142871, 119077, 80494, 105278, 12781, 25778, 131082, 26359, 97152, 128735, 36063, 149954, 46228, 94468, 52917, 29630, 78113, 36976, 56371, 4139, 157955, 106212, 46133, 112103, 128435, 1819, 40244, 156159, 78988, 118301, 135279, 121540, 68849, 10896, 120556, 72690, 118838, 130501, 27475, 157664, 7139, 57092, 155948, 38540, 36355, 103333, 130615, 66837, 64640, 88839, 80516, 104785, 154332, 78687, 125724, 143588, 158326, 46574, 81066, 64864, 51737, 27858, 19830, 106748, 53196, 100637, 70997, 128760, 121953, 40214, 115470, 124808, 71362, 145534, 147104, 106301, 107880, 68132, 31026, 98961, 84558, 53556, 84622, 76396, 36841, 143299, 155653, 17162, 159055, 52894, 43202, 78345, 21613, 152188, 116686, 5027, 100122, 92814, 116469, 146305, 10292, 155559, 2584, 103272, 58736, 152819, 51545, 65412, 140387, 77758, 56693, 8764, 17971, 31380, 41425, 73365, 75076, 154852, 30583, 15539, 7807, 49536, 134432, 50353, 115460, 92047, 44408, 44673, 124912, 82515, 85298, 98201, 18818, 134416, 4667, 136219, 79190, 80438, 103405, 141442, 28603, 88388, 22631, 154666, 35539, 21262, 13618, 12520, 32306, 49112, 75962, 12439, 140509, 115337, 34961, 52229, 72241, 93897, 130788, 61976, 27185, 106911, 83267, 147971, 141064, 139497, 105716, 8789, 120437, 149681, 73226, 143255, 95390, 59367, 887, 139989, 109572, 72950, 22397, 108105, 56638, 92922, 56562, 30071, 145819, 57510, 84161, 59037, 151330, 47809, 99789, 64669, 112015, 17010, 57238, 89687, 15867, 110174, 72006, 26926, 37923])
([7], [142033, 148894, 12107, 113883, 141794, 14823, 21059, 97764, 67539, 56727, 75530, 17964, 124972, 76404, 30946, 113835, 33358, 55603, 20450, 65943, 106262, 150944, 65444, 98859, 34281, 75356, 104781, 113667, 85332, 62309, 104642, 93816, 33705, 111024, 71718, 57874, 98925, 89274, 90054, 93139, 2372, 158716, 67020, 130593, 150962, 137694, 34774, 67319, 128247, 159339, 144930, 39043, 107151, 54303, 85524, 131178, 87554, 36330, 97016, 39866, 64846, 23736, 64667, 43859, 3894, 72210, 58512, 83986, 10954, 11123, 44616, 95102, 148857, 141994, 8975, 137317, 104199, 113250, 42501, 35370, 40860, 121277, 31150, 59709, 111757, 94760, 140430, 95741, 4917, 123700, 50888, 96936, 151027, 22560, 104788, 146859, 44233, 9338, 68168, 5541, 121479, 102064, 99718, 146272, 23728, 157482, 145197, 91281, 44945, 115375, 57197, 123149, 21561, 115767, 87537, 47015, 16352, 105838, 25882, 52122, 6961, 89437, 135900, 109488, 45106, 139506, 66112, 125886, 34986, 63012, 67729, 39007, 136026, 39581, 45936, 25589, 70625, 34460, 64159, 143219, 88318, 138813, 11463, 64516, 127441, 64525, 158218, 121036, 57387, 130520, 98025, 76409, 110077, 153818, 90211, 60010, 19306, 11885, 60005, 2380, 14682, 156628, 106938, 99426, 126048, 30643, 6541, 120367, 99881, 40460, 34237, 9222, 87328, 360, 125365, 59307, 125586, 23236, 126266, 135965, 45949, 109200, 42140, 64829, 130072, 139735, 26457, 155963, 114580, 82616, 79583, 153083, 24753, 149907, 80693, 48759, 106269, 55550, 39265, 29728, 152913, 50833, 144439, 40931, 132771, 135808, 2036, 89564, 35812, 125776, 129629, 109245, 12379, 50801, 80684, 144225, 115647, 141672, 17319, 132883, 66128, 105281, 73283, 7276, 85619, 31244, 139824, 95262, 121770, 38746, 114320, 96262, 134423, 133671, 2544, 44042, 46230, 65257, 68805, 76852, 43712, 42849, 134257, 84528, 43299, 115172, 74813, 68780, 18889, 12349, 32197, 97795, 28076, 5506, 14670, 104010, 155398, 32125, 78453, 60239, 146035, 54019, 122522, 20357, 21857, 146203, 154725, 12960, 129297, 57894, 105681, 108653, 78841, 145486, 31675, 159484, 143501, 102224, 112023, 46972, 94995, 65062, 3154, 128824, 37892, 125896, 84706, 38930, 82575, 38059, 141673, 146760, 60883, 63954, 114571, 48801, 159046, 33022, 17509, 96960, 80333, 136194, 81774, 99007, 24395, 40282, 22219, 113467, 30261, 81221, 121767, 66286, 17667, 47060, 42985, 75873, 151195, 91494, 8245, 91634, 135301, 157822, 22069, 151803, 107252, 66835, 66411, 54367, 54577, 111775, 133126, 88673, 104094, 42800, 135510, 36765, 155325, 136947, 130298, 135824, 115680, 13305, 4362, 113901, 102397, 65030, 81294, 20524, 87747, 126595, 1784, 157911, 90995, 141810, 61924, 44940, 37851, 22478, 148828, 85505, 53690, 51283, 98894, 155202, 137084, 136111, 150446, 124944, 113189, 59189, 78866, 65169, 138358, 62790, 71385, 47703, 32971, 20916, 16863, 113770, 44826, 133697, 151990, 1497, 35078, 121589, 60231, 8777, 88042, 600, 75441, 128542, 128658, 138607, 4505, 96110, 106394, 72380, 69951, 159748, 39428, 65641, 57288, 100442, 126135, 98401, 75500, 113120, 8145, 67978, 17017, 78120, 141039, 9489, 119383, 88459, 129328, 6867, 41354, 64993, 7344, 31737, 32439, 32431, 138371, 138426, 131825, 30661, 116131, 139168, 23138, 127002, 106352, 141123, 37647, 131386, 48950, 157278, 29284, 35413, 51016, 151109, 15645, 85227, 134854, 123264, 32281, 115788, 79709, 93400, 20608, 87332, 158878, 35578, 13350, 42329, 109174, 93438, 91336, 28690, 18994, 55011, 107086, 65055, 45390, 115317, 58354, 105706, 81888, 63493, 8320, 50054, 3876, 139798, 10535, 41160, 84145, 28404, 12823, 127528, 31344, 41492, 109992, 111657, 79400, 65136, 64548, 95620, 58439, 127628, 63243, 72489, 123052, 135864, 35173, 82659, 48939, 102022])
([8], [100596, 26793, 73616, 157605, 77540, 114733, 104709, 6141, 32421, 97135, 52238, 49777, 42358, 107973, 60191, 214, 48218, 121996, 25108, 17752, 86289, 89096, 66517, 102457, 75706, 29469, 114016, 91439, 9368, 20067, 55256, 150775, 64573, 13935, 126679, 46628, 128976, 104520, 11346, 153061, 43830, 69741, 82649, 112670, 63816, 29083, 42971, 40422, 150421, 92376, 112725, 103704, 87673, 110627, 32597, 48424, 116942, 63553, 62668, 145702, 119305, 9626, 37201, 158115, 152633, 80202, 43215, 152947, 13890, 56417, 37587, 110746, 41209, 30968, 94917, 77017, 107600, 106517, 84015, 31346, 42870, 24551, 2278, 134314, 49924, 141642, 29941, 121551, 88223, 22217, 31314, 113276, 63107, 75338, 104954, 99850, 135745, 52275, 2247, 25888, 24026, 111483, 111988, 19661, 17982, 69215, 151421, 131291, 134534, 137789, 137195, 92130, 4093, 15458, 147697, 13606, 32246, 21033, 34065, 76483, 58100, 52209, 10906, 92195, 135274, 136226, 37180, 58316, 78004, 132362, 137229, 94529, 24074, 90061, 114769, 35801, 68430, 46907, 75203, 112130, 25326, 34953, 85440, 142738, 65353, 113315, 54358, 22487, 78944, 143880, 72100, 43945, 73462, 129222, 8018, 32863, 54976, 53411, 90274, 49892, 83224, 10540, 140342, 98082, 73561, 113933, 14961, 73292, 105412, 102213, 104766, 102546, 80804, 139577, 59486, 123996, 121250, 66958, 67415, 105634, 133232, 95633, 5554, 35521, 13688, 91598, 91615, 77899, 100187, 50394, 136266, 41831, 38092, 141317, 71157, 155366, 135096, 153124, 128369, 113924, 31514, 15962, 25579, 18746, 52373, 159328, 156460, 153369, 98839, 130122, 97083, 56674, 111842, 55704, 8156, 146942, 14781, 40058, 145785, 142624, 46233, 151758, 758, 111117, 57799, 23195, 153284, 49081, 120553, 23558, 145580, 87983, 106620, 32416, 111454, 129031, 35336, 16656, 42132, 76126, 6740, 140440, 110193, 27378, 143760, 49975, 17720, 118075, 8136, 146789, 69955, 79916, 88385, 89411, 127699, 143620, 123736, 154528, 118175, 42783, 129729, 37141, 71492, 137137, 66298, 132682, 59505, 8073, 67429, 101180, 36522, 17461, 92943, 99607, 126554, 152020, 124627, 153706, 89147, 26049, 33097, 96904, 19248, 109951, 16125, 43823, 21310, 546, 6648, 43480, 152016, 60942, 119172, 105117, 30976, 66244, 27057, 147897, 24634, 122018, 63023, 123849, 25366, 29380, 120805, 33996, 111334, 153933, 125028, 130288, 145934, 94314, 154078, 46607, 54089, 128507, 158215, 82388, 103618, 46580, 59369, 20238, 47528, 157259, 34100, 48758, 94628, 37756, 9460, 98778, 128577, 102961, 35449, 132668, 91940, 50123, 82289, 134475, 83975, 65077, 72610, 116059, 115505, 157870, 37743, 55689, 115471, 134246, 64986, 63500, 121064, 11872, 5794, 57647, 33537, 22881, 111959, 115988, 109077, 7521, 82208, 63359, 134516, 1648, 66453, 40499, 12168, 3484, 55116, 11536, 136985, 13675, 28873, 97252, 107685, 37855, 35283, 159482, 25412, 97404, 8231, 48568, 49401, 23845, 119313, 158919, 69279, 61979, 28909, 60352, 2713, 62948, 91563, 116249, 19565, 58494, 144286, 78720, 141311, 109567, 107698, 59039, 19589, 26371, 31359, 104302, 97834, 70881, 145252, 85903, 106193, 149027, 113635, 144706, 52014, 85522, 76881, 20105, 157459, 60580, 15412, 6701, 89545, 138850, 155195, 32342, 112685, 82664, 136488, 157458, 148569, 129170, 36091, 143095, 10750, 77414, 10211, 71535, 83687, 96577, 28619, 2729, 17425, 154562, 118396, 104423, 44818, 78798, 77962, 138305, 42021, 155395, 93049, 107152, 37950, 2761, 144470, 132073, 89165, 117670, 33489, 143763, 130011, 146127, 132402, 97828, 14159, 129472, 20942])
([9], [20256, 66572, 124221, 135690, 139178, 52605, 5069, 95736, 2322, 6485, 39114, 143306, 127583, 143688, 140611, 13327, 37288, 125549, 143995, 126335, 73777, 94292, 39953, 24504, 139344, 60987, 53554, 80416, 126858, 71198, 5928, 96678, 41118, 31535, 34214, 154384, 69247, 35886, 157903, 23534, 117250, 101235, 125286, 123929, 150234, 43469, 25666, 131990, 108314, 143812, 6553, 22092, 1245, 23155, 159019, 41250, 94142, 138288, 102941, 123764, 69158, 878, 25113, 102721, 277, 41768, 107313, 156652, 158568, 62666, 63085, 120526, 26173, 145660, 28037, 105343, 61221, 77892, 154484, 138891, 65785, 29171, 11467, 94323, 96671, 12689, 115236, 132710, 159772, 41604, 96428, 33863, 70412, 88715, 99822, 115065, 131732, 152904, 53600, 76872, 83452, 54812, 13719, 143252, 128323, 62762, 81086, 5087, 65295, 54716, 48770, 62738, 62159, 98915, 147820, 121603, 77031, 108677, 140589, 141824, 22242, 14218, 111692, 20733, 45814, 42927, 104107, 44999, 82399, 153235, 96818, 135154, 130390, 94865, 44701, 44845, 45041, 58907, 4397, 146433, 159319, 25251, 24872, 95738, 120217, 139947, 83776, 75514, 86275, 105535, 108064, 29785, 37381, 17243, 73263, 106183, 98893, 157789, 93274, 28840, 1415, 103487, 121934, 36164, 78102, 76433, 51695, 40335, 116789, 17945, 144675, 108284, 6950, 128240, 141507, 91663, 92823, 89076, 133657, 52276, 3429, 84315, 27762, 111902, 119830, 79323, 72653, 66038, 120572, 10577, 87041, 109760, 84136, 57785, 42415, 51893, 6102, 29016, 93812, 109005, 97943, 11045, 127218, 19295, 82962, 8712, 117190, 35730, 12718, 13997, 45701, 106592, 62492, 148324, 98323, 95669, 51956, 130659, 33913, 38140, 68061, 99576, 145806, 91027, 3102, 78546, 146830, 100516, 15247, 90892, 92833, 58302, 27383, 23712, 63722, 10273, 27546, 67357, 9332, 60172, 154174, 106510, 64200, 75375, 71960, 13443, 111746, 68074, 141689, 57228, 152115, 129383, 39169, 53070, 82413, 142597, 43385, 49454, 95458, 7959, 72000, 99553, 55301, 12217, 62391, 37785, 53847, 105209, 93160, 107376, 48839, 74517, 674, 108118, 131087, 143015, 9239, 43266, 60822, 106375, 125259, 138726, 72074, 12098, 104462, 115860, 157125, 65320, 139663, 96517, 106958, 50522, 37556, 4096, 91019, 135202, 70508, 2874, 83484, 70401, 143191, 7094, 98106, 119537, 129320, 81558, 129691, 127665, 81490, 114699, 93479, 91337, 40674, 6599, 133621, 18778, 106438, 35547, 65352, 98584, 12353, 55387, 18561, 145497, 2425, 79995, 115331, 85023, 98906, 124749, 54833, 9680, 81023, 96432, 54943, 110313, 52486, 77173, 125840, 37540, 36192, 80279, 35051, 3278, 136466, 3298, 80588, 133372, 121956, 59190, 136479, 33320, 151664, 69727, 41233, 115063, 40572, 98478, 33078, 31858, 11232, 34154, 107836, 151901, 3469, 96785, 52602, 48406, 66149, 33979, 143591, 19632, 118882, 98263, 59387, 140723, 77964, 128190, 135915, 60379, 82194, 128419, 43730, 66881, 110074, 151384, 135909, 88745, 77224, 68813, 144438, 32734, 41197, 67778, 12851, 104563, 107242, 110773, 121152, 81697, 507, 63578, 62102, 33254, 102101, 72908, 47778, 84531, 67505, 13022, 90649, 87934, 115705, 18235, 10200, 72911, 84827, 65691, 53655, 51785, 128623, 23411, 68110, 25521, 130681, 159385, 126556, 75865, 69394, 65556, 119088, 34161, 80489, 19576, 117125, 94802, 87095, 15313, 69346, 103166, 97313, 39584, 15774, 120145, 150409, 60018, 126269, 146294, 146028, 131826, 103329, 30517, 32064, 53054, 43149, 97457])
([10], [146140, 59228, 57225, 107912, 97508, 18911, 90707, 10397, 151470, 68417, 129750, 19888, 154722, 60769, 159420, 159372, 95721, 157698, 103243, 19956, 82579, 152500, 113080, 105094, 148196, 59332, 49672, 78763, 139580, 119416, 74189, 68166, 155887, 137955, 136765, 15516, 42987, 59685, 135982, 134901, 60959, 136497, 102783, 46365, 93169, 145341, 79473, 109783, 106078, 44349, 86173, 102759, 109452, 127511, 123410, 136541, 67303, 22316, 118087, 47825, 131401, 66943, 159283, 1944, 124653, 96708, 1183, 77672, 140763, 24843, 75386, 34938, 37408, 36964, 31518, 63686, 51984, 89307, 88227, 1395, 21882, 78636, 89299, 84429, 84875, 61558, 40381, 69176, 139796, 53803, 159388, 11988, 26782, 4907, 55410, 21720, 27652, 58280, 100337, 61975, 65755, 63215, 29703, 43290, 53621, 73142, 6152, 63283, 94071, 32449, 71842, 144258, 81915, 7460, 11871, 99880, 115646, 92373, 105071, 109094, 66631, 100746, 54962, 57856, 135802, 126094, 129608, 54402, 158796, 151114, 123596, 63788, 85018, 87418, 16501, 7019, 66260, 15182, 130586, 35873, 92981, 86704, 137823, 140224, 88164, 52387, 15879, 7712, 108807, 143122, 48281, 87010, 50375, 49545, 123777, 99712, 150784, 89059, 128730, 45272, 111759, 3550, 112242, 89721, 54769, 95962, 58309, 155402, 142268, 90779, 14204, 3486, 113683, 111457, 17793, 153078, 57972, 87392, 38066, 55061, 38555, 13326, 66839, 146710, 65528, 104579, 135457, 48444, 14404, 94083, 28985, 35317, 124059, 68876, 159045, 49763, 65147, 17371, 12022, 29151, 53807, 27066, 16706, 98616, 96459, 62019, 75902, 21672, 24304, 47110, 152811, 17904, 129609, 109199, 15123, 53529, 146408, 135311, 112469, 127604, 114186, 156190, 42154, 128993, 93834, 133038, 141084, 118245, 13081, 95707, 20586, 25014, 64095, 67182, 63696, 35210, 113153, 45571, 6453, 133332, 144172, 75225, 10035, 158638, 71918, 38739, 62009, 23972, 59973, 6888, 40002, 35170, 69569, 116814, 116026, 54118, 93215, 92492, 148738, 112612, 9505, 121599, 63624, 89865, 65888, 90167, 83364, 75897, 157994, 149250, 127574, 1953, 81309, 129793, 115797, 155173, 101027, 117959, 13087, 44089, 47054, 144807, 138517, 50690, 113996, 96435, 142593, 31852, 109648, 17440, 45589, 151267, 37459, 54831, 88410, 85130, 78309, 129698, 38618, 38805, 88977, 133753, 49173, 74701, 33695, 123130, 114664, 85284, 44833, 73548, 14724, 112179, 9501, 106039, 76125, 75432, 17202, 43002, 17906, 107579, 37722, 145957, 13915, 100733, 129201, 72652, 40721, 18365, 26656, 18853, 3121, 113972, 139502, 90222, 44194, 40766, 79695, 97949, 122833, 90085, 92150, 24428, 156724, 152692, 85053, 6796, 10221, 148238, 111524, 88990, 146187, 73794, 14221, 157074, 58136, 36167, 6652, 77266, 99219, 140978, 84945, 65803, 9998, 140790, 129563, 37405, 16375, 84021, 61950, 86869, 143742, 32437, 37757, 97184, 123185, 22914, 41733, 82566, 81606, 68714, 108169, 3717, 53996, 22950, 60123, 81450, 48921, 21562, 118953, 616, 41714, 25551, 97150, 67847, 54043, 142592, 91468, 139059, 11177, 7735, 52778, 94441, 114990, 116512, 140131, 123768, 27071, 131170, 146519, 10334, 122739, 118677, 17442, 47864, 85424, 33232, 106818, 4466, 146468, 139037, 45396, 49153, 125558, 140543, 137997, 85937, 69929, 33581, 4554, 41673, 138877, 159864, 149062, 83872, 144131, 23745, 9109, 32239, 25897, 8692, 147476, 128008, 54032, 14324, 128718, 95960, 63588, 64876, 50862, 34014, 71141, 51026, 99724, 109652, 55500, 85529, 136770, 95781, 51900, 52566])
([11], [108739, 89011, 3493, 44593, 131788, 149750, 23969, 147783, 80422, 16974, 19640, 93516, 126744, 21161, 138279, 44304, 56609, 78995, 96599, 68816, 141231, 152511, 158594, 75878, 139462, 151649, 8555, 119965, 2605, 38499, 157018, 152483, 143545, 51814, 41262, 147959, 83568, 119649, 53691, 137259, 136731, 8302, 8934, 71708, 40311, 37232, 145552, 84472, 61083, 41141, 29954, 150859, 158195, 1683, 70404, 97081, 119225, 152121, 158985, 32964, 140063, 9357, 22248, 72255, 149162, 153242, 32761, 135150, 117721, 48791, 58935, 81866, 66582, 97256, 116227, 39840, 65214, 38204, 41537, 88210, 123659, 85202, 26267, 151672, 126865, 19358, 68491, 43984, 107841, 26338, 103742, 96258, 132231, 50623, 80244, 155893, 86619, 111231, 88154, 146732, 145561, 61917, 150090, 121318, 153154, 56, 116165, 80546, 45830, 59708, 19994, 33527, 156948, 57827, 25623, 128148, 20752, 91370, 158054, 107256, 125744, 46942, 158461, 123723, 98346, 58534, 36935, 65832, 28935, 38827, 132998, 30933, 88301, 62030, 108127, 88725, 131259, 108788, 148022, 55954, 157573, 23704, 86012, 159196, 51206, 151207, 57205, 118875, 36027, 140295, 90292, 80134, 71391, 51962, 80456, 80954, 83405, 128376, 113895, 71305, 13026, 140569, 71550, 37783, 16303, 119002, 42072, 81769, 134407, 119137, 41145, 5718, 65964, 88896, 144526, 57565, 156512, 95081, 107343, 53212, 85291, 6687, 91012, 99518, 19924, 142338, 126802, 139576, 11964, 120516, 27640, 94444, 131335, 49408, 124103, 105663, 74344, 49103, 52892, 95079, 64092, 44793, 107223, 111010, 51251, 80011, 137912, 48005, 156727, 70073, 13242, 12049, 54131, 26329, 13512, 89595, 138458, 121083, 105851, 69939, 89947, 141835, 27661, 152210, 101495, 10853, 29638, 121487, 31740, 14026, 49376, 37473, 8821, 154864, 157544, 155221, 34837, 144026, 36204, 70438, 92876, 93421, 61530, 25337, 91911, 70246, 79802, 155834, 95133, 138761, 61409, 148246, 98104, 89057, 73109, 135342, 157338, 154431, 85495, 86420, 83803, 50670, 1984, 135465, 118213, 116500, 90238, 69959, 129277, 103737, 125801, 70885, 27759, 41356, 114667, 7383, 26317, 48627, 57991, 94574, 93522, 18805, 112486, 126205, 63130, 41851, 151903, 87670, 36605, 151331, 56391, 145745, 75077, 109933, 81079, 139600, 90299, 57098, 125759, 70007, 101693, 142576, 115399, 139023, 24586, 128904, 119492, 110195, 48263, 154072, 83056, 73171, 114530, 18548, 116230, 158708, 55725, 148561, 136358, 77273, 61230, 81733, 55132, 42720, 629, 147020, 86302, 4205, 115153, 112791, 150513, 124320, 62753, 91711, 133188, 128403, 138799, 95577, 111286, 151858, 30971, 145242, 9896, 88672, 119553, 83824, 103073, 38384, 112543, 150576, 20631, 27070, 34253, 59112, 91436, 140001, 20959, 66073, 136074, 133650, 66707, 119773, 96207, 46711, 87949, 19634, 1135, 46783, 157111, 132194, 43103, 44184, 138205, 15727, 13637, 152915, 143477, 29927, 21419, 143419, 24639, 100531, 51730, 85366, 69269, 7977, 105007, 117037, 93386, 73071, 150726, 128825, 136851, 11247, 88464, 123129, 110067, 93141, 134533, 5432, 153331, 115969, 98206, 112585, 151499, 135729, 76018, 145937, 44322, 53532, 75066, 29332, 25808, 48285, 5884, 27341, 42356, 75174, 2460, 94732, 45267, 38876, 120180, 155563, 68916, 20186, 67407, 137321, 144876, 41637, 130500, 48456, 74619, 151834, 21943, 16342, 9317, 140866, 141241, 93081, 141269, 9283, 116171, 30897, 120228, 15851, 22049, 102315, 80911, 59700, 71537, 15039, 5060, 77944, 63388, 109999])
([12], [54213, 7297, 89357, 76931, 120851, 64515, 68598, 147130, 61168, 39931, 75700, 10665, 91681, 101882, 81347, 119377, 130250, 50607, 79945, 93906, 140348, 12477, 10501, 32270, 104128, 136535, 61369, 119096, 101918, 67304, 52781, 22275, 117815, 115600, 2417, 141802, 71153, 150434, 27296, 25485, 132547, 119254, 1240, 65303, 146303, 91625, 2431, 90696, 156294, 159273, 115842, 13228, 104799, 6914, 28704, 20710, 51386, 131235, 155111, 43294, 6593, 102162, 53868, 49100, 45233, 54844, 22490, 116602, 99587, 16088, 39233, 2342, 73949, 14198, 102618, 111814, 15038, 44460, 147766, 38418, 41392, 28026, 13580, 106796, 38260, 147080, 97151, 109422, 38287, 65182, 124004, 28510, 104918, 156307, 89613, 83052, 83621, 75061, 23328, 23164, 135431, 69338, 89104, 128476, 141088, 135051, 5772, 119532, 95657, 80622, 79675, 92323, 124034, 2097, 9158, 108323, 130321, 35992, 6789, 34599, 83588, 133925, 74042, 90578, 129349, 92901, 93776, 65634, 27239, 143041, 124436, 112671, 100028, 97425, 60595, 98454, 130283, 159053, 96914, 34654, 76598, 93928, 76644, 143881, 61505, 37488, 82467, 8782, 20129, 115373, 69815, 147882, 101420, 5828, 21538, 93860, 70753, 81180, 68895, 159180, 143392, 105538, 57046, 107683, 148145, 112917, 154677, 26076, 120053, 158727, 101909, 143269, 106931, 152866, 127779, 139640, 38607, 56109, 96433, 118982, 36469, 33378, 148731, 64059, 139360, 74817, 39618, 50334, 86625, 36735, 88217, 95792, 152429, 99429, 120080, 2537, 75803, 159379, 117012, 157552, 55920, 12737, 157779, 74666, 74658, 123461, 113943, 29714, 63522, 155389, 101413, 46982, 68884, 386, 23325, 139573, 27090, 158479, 146679, 116162, 24308, 62486, 119723, 147000, 142035, 61854, 32096, 30059, 59155, 86037, 99035, 113022, 71742, 147700, 103276, 69808, 387, 67615, 74581, 66059, 108839, 109936, 139072, 127091, 81598, 37685, 28771, 88396, 23813, 102808, 156118, 32701, 91587, 112919, 49727, 19487, 117970, 24510, 15565, 57425, 6252, 112003, 15530, 146920, 4380, 8394, 73190, 67242, 76494, 17368, 89977, 106340, 59946, 22051, 102717, 16789, 37729, 72365, 98544, 158995, 108010, 51086, 60209, 22405, 17060, 61207, 61381, 92084, 96125, 16401, 120385, 73574, 356, 50737, 30995, 116803, 5046, 4221, 51934, 109847, 143139, 5630, 130202, 4593, 47912, 123862, 82146, 52611, 152747, 156205, 32319, 82748, 120352, 148631, 99384, 92727, 70547, 68331, 3458, 140750, 104773, 79485, 94960, 5421, 24130, 149753, 150399, 108120, 129393, 132940, 127048, 74638, 21584, 154386, 91628, 69349, 106408, 95345, 19973, 83378, 50389, 68657, 73427, 137938, 26780, 51887, 3305, 15627, 69319, 130519, 135546, 47620, 18142, 53543, 73366, 127476, 154983, 100197, 121117, 148960, 27879, 96124, 10179, 6308, 69623, 26779, 155034, 112107, 45653, 82601, 16969, 91935, 123396, 37808, 76507, 71317, 18934, 98431, 13937, 89326, 50616, 125525, 91224, 128398, 40926, 94008, 35961, 8577, 115478, 102652, 109067, 40762, 26012, 41019, 55031, 112508, 12990, 38661, 148366, 89920, 151202, 135528, 83217, 137799, 135961, 75785, 30557, 25710, 42382, 96375, 118585, 15616, 91214, 137830, 150361, 79197, 45027, 70373, 80509, 117096, 16728, 136717, 119462, 140020, 124774, 113367, 56990])
([13], [97335, 98644, 42086, 7081, 25398, 13942, 141684, 5025, 115024, 100898, 159550, 159214, 60323, 149397, 60521, 36454, 106783, 109671, 49682, 34463, 119802, 68274, 22344, 32434, 107132, 86225, 146951, 92180, 2819, 85294, 24885, 9001, 85411, 11957, 134544, 4643, 23596, 132241, 101533, 9941, 44359, 84902, 54891, 98087, 57335, 109402, 132632, 15386, 36414, 73382, 118076, 75990, 10402, 22108, 99348, 19229, 119444, 150974, 61619, 84473, 16107, 138172, 139534, 1633, 155440, 121797, 140207, 47633, 75988, 95281, 99998, 74044, 136630, 117371, 82277, 138651, 74587, 45137, 152169, 123236, 123925, 155043, 11518, 153839, 99169, 41256, 140220, 36918, 7123, 63754, 146081, 94715, 93708, 157234, 100675, 99062, 103313, 79092, 37745, 4502, 77666, 87295, 159169, 82599, 17269, 8472, 11601, 11590, 136546, 132204, 5553, 61741, 67768, 77346, 145781, 81975, 118878, 62034, 54635, 9444, 16984, 99713, 37827, 20149, 64139, 73615, 90511, 94921, 24575, 84081, 146691, 91949, 50918, 19432, 23374, 86744, 17632, 147549, 60748, 75911, 54717, 156492, 88598, 140049, 53809, 27352, 97121, 87381, 109371, 68032, 29084, 47708, 50924, 48022, 56097, 7078, 103595, 4598, 78121, 69542, 38911, 26044, 152818, 95745, 131493, 154443, 150868, 92077, 31210, 45528, 55656, 146563, 138069, 130022, 131429, 159410, 31219, 98817, 73846, 73310, 6959, 94358, 88333, 71557, 154883, 22790, 23961, 68718, 114293, 41079, 129106, 37779, 111241, 133168, 112247, 33972, 98856, 27128, 122823, 125278, 38007, 125808, 104657, 94617, 26274, 158718, 74917, 58769, 155281, 72412, 79266, 71762, 72385, 150122, 98717, 68653, 102849, 134324, 95563, 5896, 8335, 44118, 61654, 34700, 126697, 130362, 29816, 57671, 41053, 13114, 41533, 21378, 139294, 75444, 40240, 135396, 117209, 118261, 141193, 120064, 116379, 31549, 27357, 125804, 96628, 134858, 10727, 150643, 152758, 86025, 20402, 10151, 141400, 107818, 45617, 61885, 97749, 24886, 39541, 36393, 136004, 77986, 93259, 130988, 102076, 67368, 74559, 45785, 70614, 118894, 28265, 20265, 38390, 152967, 19057, 118238, 116483, 9105, 58333, 129274, 152535, 39312, 99608, 114983, 50907, 12741, 136394, 12744, 159230, 25272, 26512, 97255, 26540, 97993, 14646, 118581, 88947, 43942, 145853, 92452, 43468, 24403, 84835, 75175, 7205, 58777, 122335, 27162, 32323, 85407, 41731, 46430, 12595, 140781, 108476, 149056, 96646, 13254, 10829, 122434, 12190, 97671, 56661, 130962, 118377, 54341, 5073, 5330, 34413, 138468, 138117, 93696, 100379, 141454, 154972, 6490, 53369, 83771, 81775, 138849, 70698, 54276, 95675, 116077, 44698, 6309, 104967, 29646, 60261, 27426, 131758, 99799, 18935, 72381, 151406, 113150, 39694, 125912, 70077, 39212, 6358, 147626, 8717, 130525, 151571, 13654, 114809, 134331, 50890, 127721, 37043, 42752, 43282, 125852, 147435, 50673, 24198, 126152, 22803, 2063, 142216, 2324, 34351, 91959, 85519, 17160, 95519, 97893, 89064, 108044, 122441, 133910, 21106, 20566, 5175, 157967, 23248, 148326, 19169, 154274, 19737, 17719, 84714, 27304, 99888, 124537, 67573, 127879, 7303, 37770, 110612, 51439, 111340, 5250, 133064])
([14], [64637, 133811, 120734, 52906, 134271, 28119, 97989, 150200, 103400, 53467, 128060, 151597, 2813, 120446, 9080, 96345, 8554, 43253, 45862, 36242, 99551, 142891, 20862, 36345, 3086, 42048, 108176, 102836, 148560, 122607, 31048, 102617, 19247, 4991, 64085, 98672, 105289, 116607, 61460, 63289, 93514, 140324, 128012, 29751, 121566, 138256, 92204, 93073, 114218, 12682, 72862, 60601, 65350, 140408, 89197, 81753, 63451, 155338, 145922, 92091, 123818, 35043, 448, 133830, 67074, 149434, 34199, 88391, 95743, 122815, 17968, 41352, 68828, 90957, 54813, 28669, 155814, 78909, 63411, 74679, 83374, 129179, 126733, 36065, 134377, 134028, 116224, 49983, 10427, 85214, 5030, 102610, 72331, 5761, 159312, 109181, 67030, 110196, 127266, 145754, 93743, 116977, 48364, 45399, 146925, 98878, 82559, 93581, 148272, 133965, 71374, 18074, 65177, 19104, 115792, 106145, 9651, 90909, 65422, 18598, 46697, 41799, 16765, 117884, 69082, 53296, 3590, 55070, 112442, 119333, 80880, 135035, 7433, 101555, 66249, 100639, 45594, 25019, 159058, 60593, 65730, 74424, 100966, 139857, 49133, 105567, 151482, 148140, 54624, 107887, 77783, 28063, 131582, 71962, 44522, 34086, 29845, 137003, 70532, 126214, 23688, 3474, 155747, 115700, 7181, 144019, 58608, 135871, 158966, 10319, 119754, 157087, 13341, 47525, 34693, 118049, 110236, 77567, 158805, 74237, 126194, 136777, 21235, 122827, 56345, 42365, 86963, 24737, 136821, 54070, 130211, 73460, 39566, 64454, 126310, 95725, 46779, 45463, 136849, 159134, 72258, 141966, 68514, 103084, 112384, 52575, 142119, 95801, 61167, 42378, 53992, 58778, 64507, 127752, 134495, 5387, 45592, 157183, 100374, 149548, 128711, 108138, 23757, 104784, 48306, 126058, 65009, 13749, 53311, 18215, 152732, 111363, 108246, 41374, 72253, 40509, 12698, 106865, 117936, 49403, 67534, 3673, 84392, 41227, 117443, 130927, 121256, 112844, 140456, 88544, 45015, 126916, 57522, 101036, 77592, 86520, 124310, 9580, 93186, 5526, 87961, 38949, 71753, 135764, 80174, 137087, 141055, 108799, 59712, 6393, 115952, 75063, 19046, 59883, 61419, 110200, 87872, 159806, 157546, 17179, 78255, 155564, 49520, 159107, 156155, 78266, 40879, 3284, 100065, 9666, 60919, 126075, 6767, 74055, 7167, 131036, 135306, 55472, 82956, 77689, 157118, 52424, 87956, 131511, 48828, 76589, 145875, 13790, 84219, 3842, 37998, 55891, 21148, 135640, 5611, 134005, 80721, 125181, 120426, 130877, 90194, 59906, 30425, 12447, 116416, 127541, 106634, 86516, 37601, 124283, 38645, 90699, 148305, 68982, 105386, 73284, 63536, 50099, 293, 116012, 12287, 117166, 34959, 158627, 60809, 130331, 136671, 7920, 77324, 50246, 101265, 47837, 45573, 137957, 145333, 47277, 56840, 65885, 75355, 86569, 37651, 111943, 71181, 141211, 103538, 116568, 21182, 90335, 114954, 67702, 156550, 151501, 65189, 107872, 25768, 15624, 52620, 140542, 137036, 107888, 136369, 53706, 53811, 126472, 125427, 18268, 109705, 122967, 42894, 52731, 118555, 140982, 159136, 115787, 89793, 135679, 88011, 39943, 79625, 66887, 65849, 142087, 76945, 7643, 34010, 112141, 150580, 67220, 37838])
([15], [157885, 155453, 58471, 152395, 50676, 100004, 7976, 63837, 152713, 141433, 3762, 103596, 158485, 22767, 12685, 15896, 8556, 110583, 72094, 22394, 130512, 45888, 94516, 66826, 46754, 129724, 12643, 7066, 69037, 149235, 111527, 70701, 6730, 59967, 158623, 16953, 10573, 69643, 15762, 101062, 43719, 9690, 139569, 99947, 98242, 65988, 42075, 135438, 127307, 136129, 112212, 21330, 149887, 114920, 31861, 122473, 101447, 93981, 65435, 127976, 116586, 57167, 57294, 119630, 10589, 109790, 24245, 55736, 37305, 28086, 69820, 23680, 141528, 63040, 38924, 48188, 74859, 103838, 71711, 41958, 33117, 154858, 133076, 119276, 138846, 10385, 159560, 121276, 108030, 80013, 4898, 128610, 41640, 35115, 40363, 31368, 13247, 2126, 102234, 98916, 101643, 28478, 12023, 92182, 23390, 151781, 153315, 102073, 7622, 6282, 47123, 84080, 40075, 157163, 86180, 153751, 145151, 130442, 134704, 59469, 91763, 146186, 130293, 107680, 17926, 90086, 3604, 40789, 72155, 143747, 147915, 55683, 73807, 77172, 92936, 17804, 146392, 75195, 84477, 103800, 124025, 30011, 116087, 143511, 101195, 105145, 61262, 56302, 73957, 128815, 128445, 46731, 75618, 19753, 106084, 104681, 137578, 135551, 64566, 92621, 156142, 8270, 15041, 153823, 10553, 58988, 88137, 7544, 13159, 144273, 159246, 14962, 120543, 76736, 137338, 33442, 110491, 148350, 120831, 78348, 84299, 148120, 140735, 41349, 2252, 57553, 157962, 64134, 66941, 84859, 47005, 89045, 72923, 72537, 19404, 72153, 13487, 91642, 70345, 5502, 43720, 44538, 89684, 70561, 133081, 130617, 35086, 23340, 122609, 63805, 115619, 77221, 42264, 138806, 80381, 6021, 87502, 48481, 120388, 126915, 15976, 12974, 89843, 25169, 154165, 102277, 94160, 28398, 94710, 14616, 31330, 153198, 17792, 102094, 122246, 87820, 45509, 151497, 50473, 48091, 89480, 82442, 149841, 156556, 103807, 48072, 71463, 86634, 82408, 5406, 100045, 33907, 46031, 117995, 154613, 5226, 86577, 103745, 47917, 101225, 26357, 143965, 117517, 144724, 50446, 159000, 57469, 99230, 108509, 26648, 10434, 48989, 105067, 108796, 28788, 80599, 93478, 34196, 91980, 83693, 34531, 133872, 78390, 136956, 110185, 135747, 44501, 91111, 107307, 159803, 52477, 13400, 73945, 54364, 48856, 148368, 71644, 78643, 113893, 71243, 19234, 138580, 112365, 44573, 158583, 102846, 139854, 60533, 125600, 130528, 69854, 88592, 134671, 86766, 20102, 49278, 56282, 45530, 4510, 109681, 64823, 3491, 54507, 47127, 134553, 14390, 51708, 94219, 43683, 434, 43724, 87212, 112070, 20254, 51622, 159914, 134524, 126483, 146101, 30267, 124430, 15103, 79911, 56658, 47759, 76326, 71726, 107464, 7785, 145654, 88797, 109412, 150136, 38588, 6889, 16784, 34696, 60730, 14978, 88317, 126535, 51419, 4823, 139528, 52723, 4694, 88055, 67553, 85968, 36020, 120708, 37828, 113160, 17531, 151814, 82847, 141367, 115519, 38869, 64120, 15692, 116065, 11255, 126277, 119944, 28352, 48656, 38144, 41370, 41841, 3286, 94123, 79712, 106649, 48873, 65568, 156914, 90419, 152703, 108594, 95867, 17821, 89546, 22140, 13515, 100086, 45384, 156963, 10197, 88023, 14535, 145330])
([16], [106777, 28927, 76273, 111723, 124657, 140990, 65915, 55912, 25059, 12271, 40212, 134812, 64709, 131780, 56327, 152485, 31613, 61699, 6852, 28798, 53678, 103930, 110186, 62209, 9010, 68206, 136970, 103855, 103075, 60213, 149296, 52795, 90990, 13439, 100866, 142556, 39182, 104962, 91232, 89429, 100098, 147867, 38727, 137558, 88008, 21092, 83678, 34050, 26126, 109772, 117935, 53415, 93190, 53514, 142838, 43743, 46920, 16439, 142071, 87385, 13313, 147566, 57942, 42333, 61452, 76735, 137505, 110690, 51363, 95190, 55742, 25081, 63424, 83709, 38897, 74025, 116385, 30005, 8892, 138141, 18683, 129381, 103125, 113066, 119507, 79372, 151552, 53525, 60707, 79369, 93994, 13824, 78277, 53472, 140340, 1885, 31842, 156488, 20724, 70601, 46141, 123704, 59852, 9603, 80387, 76744, 102595, 56640, 140103, 60858, 55408, 48906, 56964, 45997, 4342, 2037, 69846, 87655, 114449, 147876, 63393, 81672, 23046, 37987, 102913, 11454, 22558, 119375, 76307, 114931, 120606, 38955, 135971, 133900, 41307, 122905, 76229, 71759, 65253, 25930, 14199, 79226, 8791, 15992, 71758, 19548, 150308, 33886, 158971, 13992, 132589, 150822, 101187, 88782, 49349, 101324, 147478, 73497, 45415, 52757, 36060, 88024, 10328, 66363, 30883, 51714, 14453, 60773, 75007, 3367, 32393, 10148, 128410, 79094, 2717, 151512, 12803, 147401, 140545, 68317, 60598, 9238, 82285, 39210, 40681, 158862, 49948, 157747, 45361, 97846, 156703, 95448, 62743, 153482, 158511, 145960, 88305, 36849, 49856, 129163, 153810, 53761, 155605, 110477, 32620, 119058, 109435, 37702, 12667, 35972, 85343, 65834, 36986, 9722, 81223, 44695, 102905, 72774, 17182, 90446, 136562, 115522, 156856, 158174, 33552, 87461, 50078, 47258, 130383, 79508, 16601, 45393, 92962, 144630, 77289, 52555, 153241, 91118, 132904, 35541, 92686, 122922, 52110, 103493, 30055, 77330, 99042, 55678, 97314, 33037, 11228, 14858, 64498, 37672, 141963, 25873, 108480, 116691, 106637, 103060, 32233, 10120, 83025, 109164, 55666, 130334, 130740, 148549, 145952, 126936, 106360, 26331, 55873, 94494, 48515, 102922, 29800, 73863, 112010, 106010, 115908, 86481, 77907, 75426, 23497, 12003, 101755, 89176, 29336, 121190, 127399, 642, 56982, 70165, 69524, 142879, 94287, 18616, 14491, 39601, 42619, 8255, 158210, 77297, 105296, 46359, 72407, 127547, 13408, 114611, 98941, 27488, 41938, 85290, 120620, 117775, 150906, 157171, 64297, 153281, 16592, 43363, 97327, 135950, 92186, 93500, 100491, 90942, 131926, 71613, 104752, 10663, 89308, 14832, 46761, 149970, 7264, 102049, 82124, 138489, 139904, 114470, 6752, 14094, 109481, 95201, 19682, 8731, 100514, 46109, 75181, 94138, 146024, 124140, 117073, 21140, 90702, 140960, 19587, 152094, 66494, 2884, 3873, 31789, 154603, 32073, 130422, 114539, 96215, 30063, 123010, 150586, 156405, 152595, 154006, 80756, 53337, 47738, 85845, 153872, 134880, 29583, 102606, 132474, 126997, 124395, 86937, 34842, 18023, 52889, 461, 28074, 89538, 157616, 102196, 51366, 137464, 130386, 7894, 91146, 8845, 12314, 113917, 109271, 150314, 92818, 105317, 98785, 33625, 155135])
([17], [94768, 72801, 6529, 112239, 144190, 47386, 1636, 62865, 116074, 18320, 2623, 61956, 114668, 36756, 126436, 18760, 144651, 147501, 76062, 42160, 92617, 70981, 149145, 119193, 122998, 125922, 149368, 126751, 50928, 113260, 76240, 121885, 155191, 143582, 26655, 110722, 79655, 121124, 132461, 42893, 127543, 79992, 101809, 96867, 150584, 82221, 105596, 116189, 106466, 115698, 45067, 18175, 50269, 65707, 112845, 123232, 7527, 59763, 80162, 112097, 132797, 8258, 98409, 3516, 158107, 121747, 15677, 3142, 102347, 45160, 35105, 149486, 52440, 78574, 138153, 115975, 72987, 20310, 37353, 138718, 51046, 59381, 50561, 119057, 107032, 74448, 32885, 113523, 93982, 148840, 134768, 156440, 85929, 142844, 29820, 63709, 27381, 64486, 140859, 107504, 31489, 24072, 28672, 105204, 103837, 16473, 39935, 82225, 84836, 139447, 73434, 88779, 5830, 11442, 37623, 77160, 129949, 36950, 149118, 109720, 3869, 15852, 12166, 43885, 42541, 40144, 50108, 62232, 59049, 112325, 101430, 25236, 156136, 131502, 99611, 648, 115931, 137209, 120665, 125910, 49941, 14827, 85880, 77286, 149831, 66435, 147190, 37523, 131409, 144829, 147579, 6448, 141185, 140653, 131341, 31190, 113036, 157626, 40892, 30498, 78015, 15499, 95750, 151342, 124933, 115437, 127157, 442, 7501, 71227, 77574, 143966, 1552, 11880, 130112, 111003, 51477, 12701, 31444, 73418, 60585, 65616, 141798, 107209, 131221, 158037, 82803, 145396, 152649, 129513, 29880, 84164, 146773, 69326, 49666, 27233, 71602, 64827, 127730, 83307, 40508, 81592, 10744, 39416, 17978, 12958, 66127, 85569, 51297, 39472, 31260, 114860, 63345, 2857, 96382, 87912, 12080, 137478, 159097, 115146, 90880, 106122, 122086, 136374, 95214, 22595, 40810, 7396, 30142, 64023, 156622, 93882, 52421, 17950, 144032, 105471, 36394, 100294, 10838, 93183, 33624, 139669, 26888, 91850, 19447, 151153, 50365, 67657, 80932, 87058, 125937, 43346, 11363, 111760, 116566, 157875, 122366, 114144, 25066, 92807, 131557, 66923, 123265, 44728, 134826, 145674, 94646, 36416, 133404, 154748, 103861, 7496, 129176, 157481, 71212, 39314, 61066, 19085, 109035, 69767, 109581, 5810, 141818, 131564, 35153, 61681, 22804, 38813, 115120, 105247, 84120, 136229, 76260, 69789, 145386, 28899, 95755, 156537, 44510, 82116, 97340, 15063, 16267, 57481, 128688, 157729, 2226, 27900, 91568, 53037, 154048, 5559, 42624, 67848, 67017, 42595, 127843, 118799, 159898, 96685, 17708, 107212, 124529, 21319, 25138, 9949, 48360, 121418, 127550, 110135, 83574, 81193, 51306, 23136, 3467, 15464, 150050, 10570, 58554, 116780, 67084, 87092, 26495, 45815, 20008, 47568, 140743, 24330, 7022, 72306, 70600, 16998, 66476, 71106, 141441, 119656, 73101, 1141, 23414, 99672, 137140, 86714, 43016, 157536, 76509, 20350, 17485, 17176, 57980, 112909, 12670, 104718, 133258, 84952, 35227, 40910, 116530, 55150, 96585, 8918, 133127, 98411, 107393, 27889, 23897, 122660, 108738, 88613, 35377, 19123, 7397, 55200, 38984, 95024, 59654, 130807, 125416, 15804, 28785, 125340, 107653, 144212, 113231, 69580])
...
([1317, 1318], [159124, 151949])
([1319, 1320], [101415, 132111])
([1321, 1322], [153178, 70087])
([1323, 1324], [78916, 128078])

array([[   0.    ,  564.5804],
       [-564.5804,    0.    ],
       [ 249.0935,  813.6739]], dtype=float32)

In [7]:
1/32

0.03125

In [28]:
latent[[1,2,3]][:, np.newaxis].shape

(3, 1, 32)